In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import copy

import random
from itertools import chain, combinations, tee
import time

plt.rcParams['text.usetex'] = True

# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]

def equals(a, b, tol = 1E-3):
    if abs(a-b) <= tol:
        return True
    else:
        return False
    
def equals_array(arr_1, arr_2, tol = 1E-2):
    if np.linalg.norm(arr_1 - arr_2) <= tol:
        return True
    else:
        return False
    

In [3]:
def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

In [4]:
# arr_1 = np.array([[1, 2, 3], [4, 5, 6]])
# np.linalg.norm(arr_1)

# Download Groups, Routes to Edges Data:

In [5]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [6]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])

# Test git

In [7]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 612.539616936298,
  'Demand (eligible group, 1)': 47.77809012103125,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 27.564282762133406,
  'VoT (eligible group, 2)': 0.10238603988603988,
  'Demand (ineligible group, 1)': 115.15744798402402,
  'VoT (ineligible group, 1)': 0.2754407051282051,
  'Demand (ineligible group, 2)': 134.75871572598555,
  'VoT (ineligible group, 2)': 0.5809294871794872,
  'Demand (ineligible group, 3)': 287.28108034312373,
  'VoT (ineligible group, 3)': 1.8596449415012848},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 98.03491986336807,
  'Demand (eligible group, 1)': 7.64672374934271,
  'VoT (eligible group, 1)': 0.03197423570019724,
  'Demand (eligible group, 2)': 4.411571393851563,
  'VoT (eligible g

In [8]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [9]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

edge_to_od_dict = {}
num_edges = int(np.max(od_to_edges_array)) + 1
# print("num_edges:", num_edges)

for e in range(num_edges):
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
# od_to_edges_array
# edge_to_od_dict

In [10]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array_base = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array_base[od_index, group_index] = od_value[VoT_name]

print(demand_array)
# VoT_array_base

[[ 47.77809012  27.56428276 115.15744798 134.75871573 287.28108034]
 [  7.64672375   4.41157139  18.43056493  21.56768237  45.97837742]
 [ 88.76462936  51.21036309 213.94551691 250.3617751  533.7257842 ]
 [ 19.95836002  11.51443847  48.10476517  56.29281031 120.00603652]
 [ 75.66495318  43.65285761 182.37193845 213.41397052 454.95978261]
 [ 18.9161492   10.913163    45.59276988  53.35324135 113.73940996]
 [119.4231777   68.89798713 287.84047959 336.83460377 718.07013258]
 [ 15.23267296  22.71991899  85.716058    74.8724603   59.63978734]
 [  3.42495327   5.10840487  19.27261838  16.83451606  13.40956279]
 [ 12.98400593  19.36597495  73.06254185  63.81969017  50.83568424]
 [  3.2461412    4.84170212  18.26642165  15.95560927  12.70946808]
 [ 20.49356972  30.56668025 115.31974823 100.73110538  80.23753567]
 [  7.61874311   9.23040031  31.06102962  37.36114412  61.2429735 ]
 [ 28.88291825  34.99276635 117.75343596 141.63738759 232.1742275 ]
 [  7.2210114    8.74853304  29.43950801  35.410

In [11]:
directory_path = '../data/data_income_percentage_VoT___101_N_Sep_to_Nov_2024/'

T = 5
VoT_array = np.zeros((VoT_array_base.shape[0], VoT_array_base.shape[1], T))

for t in range(T):
    df_perturbation_data = pd.read_csv(directory_path + 'perturbations_1_' + str(t) + '.csv')
    perturbation_array = df_perturbation_data.to_numpy()[:, 1:]
    VoT_array[:, :, t] = VoT_array_base * perturbation_array
    
# VoT_array_base
# perturbation_array

# Download Latency Parameters Data

In [12]:
directory_path_latency = '../data/pems_latency_inference___101_N_Sep_to_Nov_2024/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [13]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
#     if city != "Belmont":
    if 1 == 1:
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Latency (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 861.9885,
  'Latency (at bend)': 1.326448252,
  'Slope (after bend)': 0.000782666},
 'East Palo Alto': {'Flow (at bend)': 1001.517857,
  'Latency (at bend)': 2.213126553,
  'Slope (after bend)': 0.000584484},
 'Redwood City': {'Flow (at bend)': 881.1846667,
  'Latency (at bend)': 4.892192375,
  'Slope (after bend)': 0.001563724},
 'Belmont': {'Flow (at bend)': 1278.948125,
  'Latency (at bend)': 1.199911179,
  'Slope (after bend)': 0.001994138},
 'San Mateo': {'Flow (at bend)': 1034.092826,
  'Latency (at bend)': 5.541006284,
  'Slope (after bend)': 0.002147262},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Latency (at bend)': 1.503111345,
  'Slope (after bend)': 0.000306601},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Latency (at bend)': 2.384328452,
  'Slope (after bend)': 0.000321856}}

In [14]:
# num_edges = 7
num_gp_lanes = 3

num_el = 3
num_groups = demand_array.shape[1]

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]
    

In [15]:
## Set lambdas:

lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

## Initialize tau, alpha values:

filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))

# directory_inits = '../data/opt_values___2_el_groups/'
directory_inits = '../data/opt_values___3_el_groups/'
df_inits = pd.read_csv(directory_inits + filename_segment + '___tau_B_stats_CBCP.csv')

print("filename_segment:", filename_segment)
print()

inits_tau_arr_as_object = df_inits.to_numpy()[:, 1:6]
inits_B_arr_as_object = df_inits.to_numpy()[0, 7]

argmin_tau = np.zeros((num_edges, T))
argmin_B = 0

for e in range(num_edges):
    for t in range(T):
        argmin_tau[e, t] = inits_tau_arr_as_object[e, t]
        argmin_B = inits_B_arr_as_object

print("argmin_tau:\n", argmin_tau)
print()
print("argmin_B:\n", argmin_B)


filename_segment: 1_1_1

argmin_tau:
 [[0.92 0.28 0.46 0.48 0.94]
 [0.95 0.32 0.8  0.58 0.7 ]
 [0.51 2.23 2.08 1.35 2.19]
 [1.27 0.86 1.72 1.38 2.47]
 [2.13 2.29 2.25 0.58 1.31]
 [0.53 0.44 0.69 0.5  0.38]
 [0.79 0.26 0.47 0.36 0.75]]

argmin_B:
 7.02


In [16]:
# argmin_tau = np.array([[0.0, 0.3194, 0.3194, 0.0, 0.0], \
#                        [0.2498, 0.2498, 0.2498, 0.0, 0.0], \
#                        [0.0, 0.0, 0.0, 0.9995, 0.9995], \
#                        [0.0, 1.0281, 1.0281, 1.0281, 0.0], \
#                        [1.6043, 0.0, 0.0, 1.6043, 0.0], \
#                        [0.0, 0.1922, 0.1922, 0.1922, 0.0], \
#                        [0.0, 0.0, 0.0, 0.2178, 0.2178]])

# argmin_B = 10.6925

##  <font color='blue'>Import argmin_y directly (requires storing it from the other Jupyter notebook file).</font> 

In [17]:
# argmin_y = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                     argmin_tau, argmin_B, od_to_edges_array, \
#                                     demand_array, VoT_array, num_el, coeff_input)

# print("argmin_tau:", argmin_tau)
# print("argmin_B:", argmin_B)
# # print("min_welfare:", min_welfare)
# print()
# print("argmin_y:\n")

# argmin_y


In [18]:
# ## Set lambdas:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

# directory_path = '../data/opt_values___2_el_groups/'
directory_path = '../data/opt_values___3_el_groups/'

if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
    str_int_lambda_E = str(int(lambda_E))
else:
    str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
    str_int_lambda_R = str(int(lambda_R))
else:
    str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
    str_int_lambda_I = str(int(lambda_I))
else:
    str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I

y_CBCP_opt_array = pd.read_csv(directory_path + filename_segment + '___y_CBCP.csv').to_numpy()

y_CBCP_opt = {}
for row_index in range(y_CBCP_opt_array.shape[0]):
    key = tuple(y_CBCP_opt_array[row_index,:5].astype(int))
    y_CBCP_opt[key] = y_CBCP_opt_array[row_index,-1]

# y_CBCP_array
y_CBCP_opt


{(0, 0, 0, 0, 0): 47.77806211157724,
 (0, 0, 0, 1, 0): 2.790323098426034e-05,
 (0, 1, 0, 0, 0): 27.5642548898688,
 (0, 1, 0, 1, 0): 2.781098209243989e-05,
 (0, 2, 0, 0, 0): 115.15741966796568,
 (0, 2, 0, 1, 0): 2.8060033599660888e-05,
 (0, 3, 0, 0, 0): 1.0683638630979872e-05,
 (0, 3, 0, 1, 0): 134.75870504234692,
 (0, 4, 0, 0, 0): 0.0001202670278075,
 (0, 4, 0, 1, 0): 287.28096007609594,
 (0, 0, 0, 0, 1): 47.778003802879496,
 (0, 0, 0, 1, 1): 8.621192871796957e-05,
 (0, 1, 0, 0, 1): 27.564196887727217,
 (0, 1, 0, 1, 1): 8.581312367033682e-05,
 (0, 2, 0, 0, 1): 115.15736081558724,
 (0, 2, 0, 1, 1): 8.691241202779137e-05,
 (0, 3, 0, 0, 1): 3.014180440957705e-05,
 (0, 3, 0, 1, 1): 134.75868558418114,
 (0, 4, 0, 0, 1): 0.00044106199794,
 (0, 4, 0, 1, 1): 287.2806392811258,
 (0, 0, 0, 0, 2): 47.77803662134693,
 (0, 0, 0, 1, 2): 5.33934612932228e-05,
 (0, 1, 0, 0, 2): 27.564229519371256,
 (0, 1, 0, 1, 2): 5.31814796316932e-05,
 (0, 2, 0, 0, 2): 115.15739392763196,
 (0, 2, 0, 1, 2): 5.3800367

In [19]:
# arr = np.array([11, 22, 33, 44, 55])
# tuple(arr)

# Compute Aggregated Flows, Travel Times, and Cost Metrics

In [94]:
def compute_y_agg_x(y, edge_to_od_dict, num_edges, T):
    
    y_agg = {}
    x = {}
    
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                y_agg[(e, k, t, "in")] = sum(y[(od, g, e, k, t)] \
                                                for od in edge_to_od_dict[e] for g in in_indices)
                y_agg[(e, k, t, "el")] = sum(y[(od, g, e, k, t)] \
                                                for od in edge_to_od_dict[e] for g in el_indices)

                x[(e, k, t)] = sum(y[(od, g, e, k, t)] \
                                        for od in edge_to_od_dict[e] for g in in_indices + el_indices)

    return y_agg, x


def compute_travel_times(y_agg, x, edge_to_od_dict, coeff_input, num_gp_lanes, num_edges, T):
    
    travel_times = {}
    avg_travel_times = {}
    
    for e in range(num_edges):
        for t in range(T):
            travel_times[(e, 0, t)] = coeff_input[0, e] \
                + coeff_input[1, e] * max(x[(e, 0, t)] - coeff_input[2, e], 0.0)
            travel_times[(e, 1, t)] = coeff_input[0, e] \
                + coeff_input[1, e] * max(x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)
    
    for e in range(num_edges):
        avg_travel_times[(e, 'ex')] = sum([travel_times[(e, 0, t)] for t in range(T)]) / T
        avg_travel_times[(e, 'gp')] = sum([travel_times[(e, 1, t)] for t in range(T)]) / T
        
#         print()
#         print("e:", e)
#         print("sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)):", \
#              sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)))
#         print("sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)):", \
#              sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)))
        
        avg_travel_times[(e, 'el')] = sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T))
        avg_travel_times[(e, 'in')] = sum(y_agg[(e, k, t, 'in')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
    
    avg_travel_times['ex'] = sum([travel_times[(e, 0, t)] for e in range(num_edges) for t in range(T)]) / T
    avg_travel_times['gp'] = sum([travel_times[(e, 1, t)] for e in range(num_edges) for t in range(T)]) / T

    avg_travel_times['el'] = sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for e in range(num_edges) for k in range(2) for t in range(T)) \
                                / sum(y_agg[(e, k, t, 'el')] for e in range(num_edges) for k in range(2) for t in range(T))
    avg_travel_times['in'] = sum(y_agg[(e, k, t, 'in')] * travel_times[(e, k, t)] for e in range(num_edges) for k in range(2) for t in range(T)) \
                                / sum(y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    
    return travel_times, avg_travel_times

def compute_percent_on_express(y_agg, x, num_edges, T):
    
    percent_on_express = {}
    
    for e in range(num_edges):
        percent_on_express[(e, 'el')] = sum(y_agg[(e, 0, t, 'el')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T))
        percent_on_express[(e, 'in')] = sum(y_agg[(e, 0, t, 'in')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
        percent_on_express[(e, 'all')] = sum(y_agg[(e, 0, t, 'el')] + y_agg[(e, 0, t, 'in')] for t in range(T)) \
                                        / sum(y_agg[(e, k, t, 'el')] + y_agg[(e, k, t, 'in')] for k in range(2) for t in range(T))
    
    percent_on_express['el'] = sum(y_agg[(e, 0, t, 'el')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'el')] for e in range(num_edges) for k in range(2) for t in range(T))
    percent_on_express['in'] = sum(y_agg[(e, 0, t, 'in')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    percent_on_express['all'] = sum(y_agg[(e, 0, t, 'el')] + y_agg[(e, 0, t, 'in')] for e in range(num_edges) for t in range(T)) \
                                    / sum(y_agg[(e, k, t, 'el')] + y_agg[(e, k, t, 'in')] for e in range(num_edges) for k in range(2) for t in range(T))
    
    return percent_on_express

# def compute_obj_CBCP_together(y, x, travel_times, VoT_array, tau, edge_to_od_dict, lambda_list, num_edges, T):
#     obj = {}
    
#     lambda_E, lambda_R, lambda_I = lambda_list
    
#     for e in range(num_edges):
#         obj[(e, 'E')] = sum( y[(od, g, e, k, t)] * VoT_array[od, g, t] * travel_times[(e, k, t)] \
#                             for od in edge_to_od_dict[e] for g in el_indices for k in range(2) for t in range(T) )

#         obj[(e, 'R')] = sum( y[(od, g, e, 0, t)] * tau[e, t] \
#                         for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[(e, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
#     obj['total'] = sum([obj[e] for e in range(num_edges)])
    
#     return obj


def compute_obj_CBCP(y, x, travel_times, VoT_array, tau, edge_to_od_dict, lambda_list, num_edges, T):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    for e in range(num_edges):
        for t in range(T):
            obj[(e, t, 'E')] = sum( y[(od, g, e, k, t)] * VoT_array[od, g, t] * travel_times[(e, k, t)] \
                                   for od in edge_to_od_dict[e] for g in el_indices for k in range(2))

            obj[(e, t, 'R')] = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                                   for od in edge_to_od_dict[e] for g in in_indices)
            
            obj[(e, t, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
                                   for od in edge_to_od_dict[e] for g in in_indices) \
                                + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                      for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t)] = lambda_E * obj[(e, t, 'E')] - lambda_R * obj[(e, t, 'R')] + lambda_I * obj[(e, t, 'I')]  
        
        obj[(e, 'E')] = sum([obj[(e, t, 'E')] for t in range(T)])
        obj[(e, 'I')] = sum([obj[(e, t, 'I')] for t in range(T)])
        obj[(e, 'R')] = sum([obj[(e, t, 'R')] for t in range(T)])
        
        obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
    obj['E'] = sum([obj[(e, 'E')] for e in range(num_edges)])
    obj['I'] = sum([obj[(e, 'I')] for e in range(num_edges)])
    obj['R'] = sum([obj[(e, 'R')] for e in range(num_edges)])
    
    obj['total'] = sum([obj[e] for e in range(num_edges)])
    
    return obj


In [21]:
y_agg_CBCP_opt, x_CBCP_opt \
    = compute_y_agg_x(y = y_CBCP_opt, \
                      edge_to_od_dict = edge_to_od_dict,\
                      num_edges = num_edges, \
                      T = T)

travel_times_CBCP_opt, avg_travel_times_CBCP_opt \
    = compute_travel_times(y_agg = y_agg_CBCP_opt, \
                           x = x_CBCP_opt, \
                           edge_to_od_dict = edge_to_od_dict,\
                           coeff_input = coeff_input, \
                           num_gp_lanes = num_gp_lanes,\
                           num_edges = num_edges, \
                           T = T)

percent_on_express_CBCP_opt \
    = compute_percent_on_express(y_agg = y_agg_CBCP_opt,\
                                 x = x_CBCP_opt, \
                                 num_edges = num_edges, \
                                 T = T)

lambda_list = [lambda_E, lambda_R, lambda_I]

# obj_CBCP_opt_together \
#     = compute_obj_CBCP_together(y = y_CBCP_opt, \
#                        x = x_CBCP_opt, \
#                        travel_times = travel_times_CBCP_opt, \
#                        VoT_array = VoT_array, \
#                        tau = argmin_tau, \
#                        edge_to_od_dict = edge_to_od_dict, \
#                        lambda_list = lambda_list, \
#                        num_edges = num_edges, \
#                        T = T)

obj_CBCP_opt = compute_obj_CBCP(y = y_CBCP_opt, \
                       x = x_CBCP_opt, \
                       travel_times = travel_times_CBCP_opt, \
                       VoT_array = VoT_array, \
                       tau = argmin_tau, \
                       edge_to_od_dict = edge_to_od_dict, \
                       lambda_list = lambda_list, \
                       num_edges = num_edges, \
                       T = T)


e: 0
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 12411.931160197815
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 7538.801131763732

e: 1
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 22302.473507836905
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 8834.409086385167

e: 2
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 70843.47494693559
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 11490.113254716762

e: 3
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 17607.343997826403
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 9951.409385957952

e: 4
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 96899.19190881196
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 13178.963416214334

e: 5

In [22]:
# for e in range(num_edges):
#     print("Edge:", e)
#     assert abs(obj_CBCP_opt_together[(e, 'E')] - obj_CBCP_opt[(e, 'E')]) <= 1E-3
#     assert abs(obj_CBCP_opt_together[(e, 'R')] - obj_CBCP_opt[(e, 'R')]) <= 1E-3
#     assert abs(obj_CBCP_opt_together[(e, 'I')] - obj_CBCP_opt[(e, 'I')]) <= 1E-3

In [23]:
# y_CBCP_opt

In [24]:
# y_agg_CBCP_opt
# x_CBCP_opt

# travel_times_CBCP_opt
# avg_travel_times_CBCP_opt
# percent_on_express_CBCP
obj_CBCP_opt


{(0, 0, 'E'): 495.1906551849078,
 (0, 0, 'R'): 361.3193571562317,
 (0, 0, 'I'): 8485.747633397628,
 (0, 0): 8619.618931426305,
 (0, 1, 'E'): 454.18973611145964,
 (0, 1, 'R'): 149.44334102065687,
 (0, 1, 'I'): 7635.60314340771,
 (0, 1): 7940.3495384985135,
 (0, 2, 'E'): 456.0534985435468,
 (0, 2, 'R'): 318.9823872563723,
 (0, 2, 'I'): 7804.986432836546,
 (0, 2): 7942.05754412372,
 (0, 3, 'E'): 428.1475657556268,
 (0, 3, 'R'): 367.8536012314232,
 (0, 3, 'I'): 7986.591979407653,
 (0, 3): 8046.885943931856,
 (0, 4, 'E'): 509.5178745855066,
 (0, 4, 'R'): 266.67835247210246,
 (0, 4, 'I'): 8893.417178784477,
 (0, 4): 9136.25670089788,
 (0, 'E'): 2343.0993301810477,
 (0, 'I'): 40806.34636783401,
 (0, 'R'): 1464.2770391367865,
 0: 41685.168658878276,
 (1, 0, 'E'): 931.3762335853235,
 (1, 0, 'R'): 0.0015081260626418772,
 (1, 0, 'I'): 12324.34850817442,
 (1, 0): 13255.723233633682,
 (1, 1, 'E'): 786.5458496090615,
 (1, 1, 'R'): 170.7924963753395,
 (1, 1, 'I'): 10990.668608264601,
 (1, 1): 11606.4

In [25]:
## For reference, from above:

# compute_y_agg_x_CBCP(y, edge_to_od_dict, num_edges, T)
# compute_travel_times_CBCP(y_agg, x, edge_to_od_dict, coeff_input, num_gp_lanes, num_edges, T)
# compute_percent_on_express_CBCP(y_agg, x, num_edges, T)
# compute_obj_CBCP(y, x, travel_times, VoT_array, tau, edge_to_od_dict, lambda_list, num_edges, T)


## Store into opt data array:

In [26]:
opt_data_array = np.zeros((num_edges, 16))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau, axis=1)

# argmin_B (as array)
opt_data_array[:, 6] = argmin_B * np.eye(1, num_edges, 0)

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 7] = np.array([percent_on_express_CBCP_opt[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 8] = np.array([percent_on_express_CBCP_opt[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 9] = np.array([percent_on_express_CBCP_opt[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 10] = np.array([avg_travel_times_CBCP_opt[(e, 'ex')] for e in range(num_edges)])
opt_data_array[:, 11] = np.array([avg_travel_times_CBCP_opt[(e, 'gp')] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 12] = np.array([obj_CBCP_opt[(e, 'E')] for e in range(num_edges)]) 
opt_data_array[:, 13] = np.array([obj_CBCP_opt[(e, 'I')] for e in range(num_edges)]) 
opt_data_array[:, 14] = np.array([obj_CBCP_opt[(e, 'R')] for e in range(num_edges)]) 
opt_data_array[:, 15] = np.array([obj_CBCP_opt[e] for e in range(num_edges)]) 


opt_data_array = np.round(opt_data_array, decimals=2)

In [27]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)", \
                 "B", \
                 "% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_save

,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),B,% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.92,0.28,0.46,0.48,0.94,0.62,7.02,17.26,20.07,15.99,1.40,1.70,2343.10,40806.35,1464.28,41685.17
e=2,0.95,0.32,0.80,0.58,0.70,0.67,0.00,9.58,7.73,10.54,2.22,2.54,4255.67,59361.06,985.71,62631.02
e=3,0.51,2.23,2.08,1.35,2.19,1.67,0.00,19.22,26.92,15.29,5.56,6.37,13474.17,194336.74,5489.56,202321.35
e=4,1.27,0.86,1.72,1.38,2.47,1.54,0.00,17.06,32.94,9.27,1.23,1.98,3300.30,53514.71,2676.14,54138.87
e=5,2.13,2.29,2.25,0.58,1.31,1.71,0.00,21.32,52.50,5.84,6.96,7.79,18203.18,270214.63,2201.90,286215.91
e=6,0.53,0.44,0.69,0.50,0.38,0.51,0.00,15.81,33.85,6.95,1.57,1.82,3630.42,53187.31,817.44,56000.28
e=7,0.79,0.26,0.47,0.36,0.75,0.53,0.00,16.65,43.71,3.38,2.49,2.75,5952.93,85533.54,365.20,91121.27


In [29]:
# random_string

## Initialize DBCP flows

In [30]:
# Function for transforming arrays of size (n, ) into arrays of size (n, 1)

def pad_dim(arr_or_list):
    arr = np.array(arr_or_list)
    assert len(arr.shape) == 1, "We must have len(arr.shape) == 1 to proceed"
    arr_len = arr.shape[0]
    return arr.reshape((arr_len, 1))

# Function for filling a vector from the bottom up to some value:

def fill_from_bottom(arr_or_list, val):
    arr = np.array(arr_or_list)
    assert np.all(arr >= -1E-3), "We must have all entries of arr >= 0.0."
    assert val >= -1E-3, "We must have val >= 0.0."
    
    arr_fill_from_bottom = np.zeros(arr.shape)
    
    index_boundary = max([index for index in range(arr.shape[0]) if np.sum(arr[index:]) >= val - 1E-3])
    
    for index in range(arr.shape[0]):
        if index > index_boundary:
            arr_fill_from_bottom[index] = arr[index]
        elif index < index_boundary:
            arr_fill_from_bottom[index] = 0.0
        else:
            arr_fill_from_bottom[index] = arr[index_boundary] - (np.sum(arr[index_boundary:]) - val)
            
        if abs(arr_fill_from_bottom[index]) <= 1E-3:
            arr_fill_from_bottom[index] = 0.0
            
#         print("index:", index)
#         print("index_boundary:", index_boundary)
#         print("arr_fill_from_bottom[index]", arr_fill_from_bottom[index])
            
        assert arr_fill_from_bottom[index] >= 0.0, "We must have arr_fill_from_bottom[index] >= 0.0"

    assert abs(np.sum(arr_fill_from_bottom) - val) <= 1E-3, \
        "We must have np.sum(arr_fill_from_bottom) == val"
    
    return arr_fill_from_bottom, index_boundary


In [31]:
# dict_VoTs_demands_annotated = {}

# directory_path = "../data/VoTs_demands_sorted___2_el_groups/"

# e, t = 0, 0
# filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
# df_data_in = pd.read_csv(directory_path + filename_in)
# dict_VoTs_demands_annotated[(e, t, "in")] = df_data_in.to_numpy()

# dict_VoTs_demands_annotated[(e, t, "in")]

In [32]:
start_time = time.time()

dict_VoTs_demands_annotated = {}

# directory_path = "../data/VoTs_demands_sorted___2_el_groups/"
directory_path = "../data/VoTs_demands_sorted___3_el_groups/"

for e in range(num_edges):
    for t in range(T):
        
        # Ineligible users:
        
        filename_in = str(e) + "_" + str(t) + "_" + "in.csv"
        df_data_in = pd.read_csv(directory_path + filename_in)
        dict_VoTs_demands_annotated[(e, t, "in")] = df_data_in.to_numpy()
        
        # Eligible users:
        
        filename_el = str(e) + "_" + str(t) + "_" + "el.csv"
        df_data_el = pd.read_csv(directory_path + filename_el)
        dict_VoTs_demands_annotated[(e, t, "el")] = df_data_el.to_numpy()

end_time = time.time()
print("Time:", end_time - start_time)

Time: 0.07720184326171875


In [33]:
# dict_VoTs_demands_annotated

In [34]:
# def y_agg_to_y_DBCP_full(y_agg, dict_VoTs_demands_annotated, num_edges, T):
    
#     y_DBCP = {}
#     VoT_in_boundary = np.zeros((num_edges, T))
#     VoT_el_boundary = np.zeros((num_edges, T))
#     alpha = np.zeros((num_edges, T))

#     for e in range(num_edges):
#         for t in range(T):

#             ## Ineligible user flows:

#             VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")]        
#             demands_annotated_in_ex_DBCP, index_VoT_in_boundary \
#                 = fill_from_bottom(VoTs_demands_annotated_in[:, 1], y_agg[(e, 0, t, 'in')])
#             demands_annotated_in_gp_DBCP \
#                 = VoTs_demands_annotated_in[:, 1] - demands_annotated_in_ex_DBCP

#             assert np.all(demands_annotated_in_ex_DBCP) >= 0.0, "We must have demands_annotated_in_ex_DBCP >= 0.0"
#             assert np.all(demands_annotated_in_gp_DBCP) >= 0.0, "We must have demands_annotated_in_gp_DBCP >= 0.0"

#             for row_index in range(VoTs_demands_annotated_in.shape[0]):
#                 od, g = VoTs_demands_annotated_in[row_index, 2:]
#                 od, g = int(od), int(g)
#                 y_DBCP[(od, g, e, 0, t)] = demands_annotated_in_ex_DBCP[row_index]
#                 y_DBCP[(od, g, e, 1, t)] = demands_annotated_in_gp_DBCP[row_index]

#             ## Eligible user flows:

#             VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")]        
#             demands_annotated_el_ex_DBCP, index_VoT_el_boundary \
#                 = fill_from_bottom(VoTs_demands_annotated_el[:, 1], y_agg[(e, 0, t, 'el')])
#             demands_annotated_el_gp_DBCP \
#                 = VoTs_demands_annotated_el[:, 1] - demands_annotated_el_ex_DBCP

#             assert np.all(demands_annotated_el_ex_DBCP) >= 0.0, "We must have demands_annotated_el_ex_DBCP >= 0.0"
#             assert np.all(demands_annotated_el_gp_DBCP) >= 0.0, "We must have demands_annotated_el_gp_DBCP >= 0.0"

#             for row_index in range(VoTs_demands_annotated_el.shape[0]):
#                 od, g = VoTs_demands_annotated_el[row_index, 2:]
#                 od, g = int(od), int(g)
#                 y_DBCP[(od, g, e, 0, t)] = demands_annotated_el_ex_DBCP[row_index]
#                 y_DBCP[(od, g, e, 1, t)] = demands_annotated_el_gp_DBCP[row_index]
                
#             ## Compute alpha:
        
#             VoT_in_boundary[e, t] = VoTs_demands_annotated_in[index_VoT_in_boundary, 0]
#             VoT_el_boundary[e, t] = VoTs_demands_annotated_el[index_VoT_el_boundary, 0]
#             alpha[e, t] = 1 - VoT_el_boundary[e, t] / VoT_in_boundary[e, t]
        
#             assert 0.0 <= alpha[e, t] <= 1.0, "We must have 0.0 <= alpha[e, t] <= 1.0."
    
#     return y_DBCP, VoT_in_boundary, VoT_el_boundary, alpha



In [35]:
# y_DBCP_init, VoT_in_boundary, VoT_el_boundary, init_alpha \
#     = y_agg_to_y_DBCP(y_agg = y_agg_CBCP_opt, \
#                       dict_VoTs_demands_annotated = dict_VoTs_demands_annotated, \
#                       num_edges = num_edges, \
#                       T = T)

In [36]:
# # dict_1 = {'a': 1, 'b': 2}
# dict_1 = {}
# dict_2 = {'c': 11, 'd': 22}

# dict_1 = {**dict_1, **dict_2} 
# dict_1

In [37]:
def y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t, y_agg_el_at_e_t, \
                           VoTs_demands_annotated_in, VoTs_demands_annotated_el, \
                           e, t):
    
    y_DBCP_at_e_t = {}
    
    ## Ineligible user flows:

#     VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")]        
    demands_annotated_in_ex_DBCP, index_VoT_in_boundary \
        = fill_from_bottom(VoTs_demands_annotated_in[:, 1], y_agg_in_at_e_t)
    demands_annotated_in_gp_DBCP \
        = VoTs_demands_annotated_in[:, 1] - demands_annotated_in_ex_DBCP

    assert np.all(demands_annotated_in_ex_DBCP) >= 0.0, "We must have demands_annotated_in_ex_DBCP >= 0.0"
    assert np.all(demands_annotated_in_gp_DBCP) >= 0.0, "We must have demands_annotated_in_gp_DBCP >= 0.0"

    for row_index in range(VoTs_demands_annotated_in.shape[0]):
        od, g = VoTs_demands_annotated_in[row_index, 2:]
        od, g = int(od), int(g)
        y_DBCP_at_e_t[(od, g, e, 0, t)] = demands_annotated_in_ex_DBCP[row_index]
        y_DBCP_at_e_t[(od, g, e, 1, t)] = demands_annotated_in_gp_DBCP[row_index]

    ## Eligible user flows:

#     VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")]        
    demands_annotated_el_ex_DBCP, index_VoT_el_boundary \
        = fill_from_bottom(VoTs_demands_annotated_el[:, 1], y_agg_el_at_e_t)
    demands_annotated_el_gp_DBCP \
        = VoTs_demands_annotated_el[:, 1] - demands_annotated_el_ex_DBCP

    assert np.all(demands_annotated_el_ex_DBCP) >= 0.0, "We must have demands_annotated_el_ex_DBCP >= 0.0"
    assert np.all(demands_annotated_el_gp_DBCP) >= 0.0, "We must have demands_annotated_el_gp_DBCP >= 0.0"

    for row_index in range(VoTs_demands_annotated_el.shape[0]):
        od, g = VoTs_demands_annotated_el[row_index, 2:]
        od, g = int(od), int(g)
        y_DBCP_at_e_t[(od, g, e, 0, t)] = demands_annotated_el_ex_DBCP[row_index]
        y_DBCP_at_e_t[(od, g, e, 1, t)] = demands_annotated_el_gp_DBCP[row_index]

    ## Compute alpha:

    VoT_in_boundary = VoTs_demands_annotated_in[index_VoT_in_boundary, 0]
    VoT_el_boundary = VoTs_demands_annotated_el[index_VoT_el_boundary, 0]
    alpha = 1 - VoT_el_boundary / VoT_in_boundary

    assert 0.0 <= alpha <= 1.0, "We must have 0.0 <= alpha <= 1.0."
    
    return y_DBCP_at_e_t, VoT_in_boundary, VoT_el_boundary, alpha


In [38]:
# y_DBCP_init___from_func = {}
# init_alpha___from_func = np.zeros((num_edges, T))
# VoT_in_boundary___from_func = np.zeros((num_edges, T))
# VoT_el_boundary___from_func = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
        
#         y_DBCP_init_at_e_t, VoT_in_boundary___from_func[e, t], VoT_el_boundary___from_func[e, t], init_alpha___from_func[e, t] \
#             = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'in')], \
#                                      y_agg_el_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'el')], \
#                                      VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
#                                      VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
#                                      e = e, \
#                                      t = t)
#         y_DBCP_init___from_func = {**y_DBCP_init___from_func, **y_DBCP_init_at_e_t}
        
#         assert 0.0 <= init_alpha___from_func[e, t] <= 1.0, "We must have 0.0 <= init_alpha___from_func[e, t] <= 1.0."
        
#         # y_agg_CBCP_opt[(e, 0, t, 'el')]


In [39]:
y_DBCP_init = {}
init_alpha = np.zeros((num_edges, T))
VoT_in_boundary = np.zeros((num_edges, T))
VoT_el_boundary = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
        
        y_DBCP_init_at_e_t, VoT_in_boundary[e, t], VoT_el_boundary[e, t], init_alpha[e, t] \
            = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'in')], \
                                     y_agg_el_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'el')], \
                                     VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
                                     VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
                                     e = e, \
                                     t = t)
        y_DBCP_init = {**y_DBCP_init, **y_DBCP_init_at_e_t}
        
        assert 0.0 <= init_alpha[e, t] <= 1.0, "We must have 0.0 <= init_alpha[e, t] <= 1.0."
        
        # y_agg_CBCP_opt[(e, 0, t, 'el')]


In [40]:
# y_DBCP_init = {}
# init_alpha = np.zeros((num_edges, T))
# VoT_in_boundary = np.zeros((num_edges, T))
# VoT_el_boundary = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
        
#         ## Ineligible user flows:
        
#         VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")]        
#         demands_annotated_in_ex_DBCP, index_VoT_in_boundary \
#             = fill_from_bottom(VoTs_demands_annotated_in[:, 1], y_agg_CBCP_opt[(e, 0, t, 'in')])
#         demands_annotated_in_gp_DBCP \
#             = VoTs_demands_annotated_in[:, 1] - demands_annotated_in_ex_DBCP
        
#         assert np.all(demands_annotated_in_ex_DBCP) >= 0.0, "We must have demands_annotated_in_ex_DBCP >= 0.0"
#         assert np.all(demands_annotated_in_gp_DBCP) >= 0.0, "We must have demands_annotated_in_gp_DBCP >= 0.0"
        
#         for row_index in range(VoTs_demands_annotated_in.shape[0]):
#             od, g = VoTs_demands_annotated_in[row_index, 2:]
#             od, g = int(od), int(g)
#             y_DBCP_init[(od, g, e, 0, t)] = demands_annotated_in_ex_DBCP[row_index]
#             y_DBCP_init[(od, g, e, 1, t)] = demands_annotated_in_gp_DBCP[row_index]
        
#         ## Eligible user flows:
        
#         VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")]        
#         demands_annotated_el_ex_DBCP, index_VoT_el_boundary \
#             = fill_from_bottom(VoTs_demands_annotated_el[:, 1], y_agg_CBCP_opt[(e, 0, t, 'el')])
#         demands_annotated_el_gp_DBCP \
#             = VoTs_demands_annotated_el[:, 1] - demands_annotated_el_ex_DBCP
        
#         assert np.all(demands_annotated_el_ex_DBCP) >= 0.0, "We must have demands_annotated_el_ex_DBCP >= 0.0"
#         assert np.all(demands_annotated_el_gp_DBCP) >= 0.0, "We must have demands_annotated_el_gp_DBCP >= 0.0"
        
#         for row_index in range(VoTs_demands_annotated_el.shape[0]):
#             od, g = VoTs_demands_annotated_el[row_index, 2:]
#             od, g = int(od), int(g)
#             y_DBCP_init[(od, g, e, 0, t)] = demands_annotated_el_ex_DBCP[row_index]
#             y_DBCP_init[(od, g, e, 1, t)] = demands_annotated_el_gp_DBCP[row_index]
            
#         ## Compute alpha:
        
#         VoT_in_boundary[e, t] = VoTs_demands_annotated_in[index_VoT_in_boundary, 0]
#         VoT_el_boundary[e, t] = VoTs_demands_annotated_el[index_VoT_el_boundary, 0]
#         init_alpha[e, t] = 1 - VoT_el_boundary[e, t] / VoT_in_boundary[e, t]
        
#         assert 0.0 <= init_alpha[e, t] <= 1.0, "We must have 0.0 <= init_alpha[e, t] <= 1.0."
        
#         # y_agg_CBCP_opt[(e, 0, t, 'el')]
    

In [41]:
# # y_DBCP_init___from_func
# # init_alpha___from_func
# # VoT_in_boundary___from_func
# # VoT_el_boundary___from_func

# assert y_DBCP_init___from_func.keys() == y_DBCP_init.keys()

# for key in y_DBCP_init.keys():
#     assert equals(y_DBCP_init___from_func[key], y_DBCP_init[key])
    
# assert equals_array(init_alpha___from_func, init_alpha)
# assert equals_array(VoT_in_boundary___from_func, VoT_in_boundary)
# assert equals_array(VoT_el_boundary___from_func, VoT_el_boundary)



In [42]:
# # y_DBCP___from_func, VoT_in_boundary___from_func, VoT_el_boundary___from_func, alpha___from_func

# assert y_DBCP_init___from_func.keys() == y_DBCP_init.keys()

# for key in y_DBCP_init___from_func.keys():
#     print("key:", key)
#     assert equals(y_DBCP_init___from_func[key], y_DBCP_init[key])

# for e in range(num_edges):
#     for t in range(T):
#         print()
#         print("e:", e)
#         print("t:", t)
#         print()
        
#         assert equals(VoT_in_boundary___from_func[e, t], VoT_in_boundary[e, t])
#         assert equals(VoT_el_boundary___from_func[e, t], VoT_el_boundary[e, t])
#         assert equals(init_alpha___from_func[e, t], init_alpha[e, t])

# # equals_array()

In [43]:
# list(init_DBCP_flows.keys())
# list(y_CBCP_opt.keys())

# y_DBCP_init

In [44]:
# demand_array.shape
# VoT_array.shape

In [45]:
def y_boundary_toggle_setup(y, demand_array, VoT_array, el_indices, in_indices, \
                            VoT_el_boundary, VoT_in_boundary, num_edges, T):
        
    y_agg_boundary_to_toggle = {}
    y_agg_el_ex_low = np.zeros((num_edges, T))
    y_agg_el_ex_high = np.zeros((num_edges, T))

    y_agg_el_at_boundary_VoT = np.zeros((num_edges, T))
    y_agg_in_at_boundary_VoT = np.zeros((num_edges, T))
    demand_el_at_boundary_VoT = np.zeros((num_edges, T))
    demand_in_at_boundary_VoT = np.zeros((num_edges, T))
    y_agg_el_above_boundary_VoT = np.zeros((num_edges, T))
    y_agg_in_above_boundary_VoT = np.zeros((num_edges, T))
    y_agg_at_boundary_VoT = np.zeros((num_edges, T))

    for e in range(num_edges):
        for t in range(T):
            print()
            print("e:", e)
            print("t:", t)
            print()

            y_agg_el_at_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
                       if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
            y_agg_in_at_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
                       if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
            y_agg_at_boundary_VoT[e, t] = y_agg_el_at_boundary_VoT[e, t] + y_agg_in_at_boundary_VoT[e, t]

    #         y_agg_at_boundary_VoT[e, t] \
    #             = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
    #                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3]) \
    #                 + sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
    #                        if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])

            demand_el_at_boundary_VoT[e, t] \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
                       if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
            demand_in_at_boundary_VoT[e, t] \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
                       if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])

            if abs(y_agg_el_at_boundary_VoT[e, t] - demand_el_at_boundary_VoT[e, t]) <= 1E-3 and \
                    abs(y_agg_in_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t]) <= 1E-3:
                y_agg_boundary_to_toggle[(e, t)] = False
            else:
                y_agg_boundary_to_toggle[(e, t)] = True

            y_agg_el_above_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
                       if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])

            y_agg_in_above_boundary_VoT[e, t] \
                = sum([y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
                       if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])

            demand_el_above_boundary_VoT \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
                       if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])
            demand_in_above_boundary_VoT \
                = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
                       if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])

            # print("y_agg_el_above_boundary_VoT[e, t]:", y_agg_el_above_boundary_VoT[e, t])
            # print("demand_el_above_boundary_VoT:", demand_el_above_boundary_VoT)
            # print("y_agg_in_above_boundary_VoT[e, t]:", y_agg_in_above_boundary_VoT[e, t])
            # print("demand_in_above_boundary_VoT:", demand_in_above_boundary_VoT)

            y_agg_el_ex_low[e, t] = demand_el_above_boundary_VoT \
                + max(y_agg_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t], 0.0)

            y_agg_el_ex_high[e, t] = demand_el_above_boundary_VoT \
                + min(y_agg_at_boundary_VoT[e, t], demand_el_at_boundary_VoT[e, t])
            
#             print("e:", e)
#             print("t:", t)
            
            print("y_agg_el_above_boundary_VoT[e, t]:", y_agg_el_above_boundary_VoT[e, t])
            print("demand_el_above_boundary_VoT:", demand_el_above_boundary_VoT)
            
            print("y_agg_in_above_boundary_VoT[e, t]:", y_agg_in_above_boundary_VoT[e, t])
            print("demand_in_above_boundary_VoT:", demand_in_above_boundary_VoT)

            assert y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t] + 1E-3, \
                "We must have y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t]."

            assert abs(y_agg_el_above_boundary_VoT[e, t] - demand_el_above_boundary_VoT) <= 1E-3, \
                "We should have y_agg_el_above_boundary_VoT[e, t] = demand_el_above_boundary_VoT"

            assert abs(y_agg_in_above_boundary_VoT[e, t] - demand_in_above_boundary_VoT) <= 1E-3, \
                "We should have y_agg_in_above_boundary_VoT[e, t] = demand_in_above_boundary_VoT"
    
    return y_agg_boundary_to_toggle, y_agg_el_ex_low, y_agg_el_ex_high


In [46]:
# # Old code which executes the same functionalities as the above function definition:

# y_agg_boundary_to_toggle = {}
# y_agg_el_ex_low = np.zeros((num_edges, T))
# y_agg_el_ex_high = np.zeros((num_edges, T))

# y_agg_el_at_boundary_VoT = np.zeros((num_edges, T))
# y_agg_in_at_boundary_VoT = np.zeros((num_edges, T))
# demand_el_at_boundary_VoT = np.zeros((num_edges, T))
# demand_in_at_boundary_VoT = np.zeros((num_edges, T))
# y_agg_el_above_boundary_VoT = np.zeros((num_edges, T))
# y_agg_in_above_boundary_VoT = np.zeros((num_edges, T))
# y_agg_at_boundary_VoT = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
# #         print()
# #         print("e:", e)
# #         print("t:", t)
# #         print()

#         y_agg_el_at_boundary_VoT[e, t] \
#             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
#                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
#         y_agg_in_at_boundary_VoT[e, t] \
#             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
#                    if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
#         y_agg_at_boundary_VoT[e, t] = y_agg_el_at_boundary_VoT[e, t] + y_agg_in_at_boundary_VoT[e, t]
        
# #         y_agg_at_boundary_VoT[e, t] \
# #             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
# #                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3]) \
# #                 + sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
# #                        if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
        
#         demand_el_at_boundary_VoT[e, t] \
#             = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
#                    if abs(VoT_array[od, g, t] - VoT_el_boundary[e, t]) <= 1E-3])
#         demand_in_at_boundary_VoT[e, t] \
#             = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
#                    if abs(VoT_array[od, g, t] - VoT_in_boundary[e, t]) <= 1E-3])
        
#         if abs(y_agg_el_at_boundary_VoT[e, t] - demand_el_at_boundary_VoT[e, t]) <= 1E-3 and \
#                 abs(y_agg_in_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t]) <= 1E-3:
#             y_agg_boundary_to_toggle[(e, t)] = False
#         else:
#             y_agg_boundary_to_toggle[(e, t)] = True
        
#         y_agg_el_above_boundary_VoT[e, t] \
#             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices \
#                    if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])
        
#         y_agg_in_above_boundary_VoT[e, t] \
#             = sum([y_DBCP_init[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in in_indices \
#                    if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])
        
#         demand_el_above_boundary_VoT \
#             = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in el_indices \
#                    if VoT_array[od, g, t] > VoT_el_boundary[e, t] + 1E-3])
#         demand_in_above_boundary_VoT \
#             = sum([demand_array[od, g] for od in edge_to_od_dict[e] for g in in_indices \
#                    if VoT_array[od, g, t] > VoT_in_boundary[e, t] + 1E-3])
        
#         # print("y_agg_el_above_boundary_VoT[e, t]:", y_agg_el_above_boundary_VoT[e, t])
#         # print("demand_el_above_boundary_VoT:", demand_el_above_boundary_VoT)
#         # print("y_agg_in_above_boundary_VoT[e, t]:", y_agg_in_above_boundary_VoT[e, t])
#         # print("demand_in_above_boundary_VoT:", demand_in_above_boundary_VoT)
        
#         y_agg_el_ex_low[e, t] = demand_el_above_boundary_VoT \
#             + max(y_agg_at_boundary_VoT[e, t] - demand_in_at_boundary_VoT[e, t], 0.0)

#         y_agg_el_ex_high[e, t] = demand_el_above_boundary_VoT \
#             + min(y_agg_at_boundary_VoT[e, t], demand_el_at_boundary_VoT[e, t])
        
#         assert y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t] + 1E-3, \
#             "We must have y_agg_el_ex_low[e, t] <= y_agg_el_ex_high[e, t]."
        
#         assert abs(y_agg_el_above_boundary_VoT[e, t] - demand_el_above_boundary_VoT) <= 1E-3, \
#             "We should have y_agg_el_above_boundary_VoT[e, t] = demand_el_above_boundary_VoT"
        
#         assert abs(y_agg_in_above_boundary_VoT[e, t] - demand_in_above_boundary_VoT) <= 1E-3, \
#             "We should have y_agg_in_above_boundary_VoT[e, t] = demand_in_above_boundary_VoT"
        

In [47]:
y_agg_boundary_to_toggle, y_agg_el_ex_low, y_agg_el_ex_high \
    = y_boundary_toggle_setup(y = y_DBCP_init, \
                                 demand_array = demand_array, \
                                 VoT_array = VoT_array, \
                                 el_indices = el_indices, \
                                 in_indices = in_indices, \
                                 VoT_el_boundary = VoT_el_boundary, \
                                 VoT_in_boundary = VoT_in_boundary, \
                                 num_edges = num_edges, \
                                 T = T)

# for e in range(num_edges):
#     for t in range(T):
#         print()
#         print("e:", e)
#         print("t:", t)
#         print()
        
#         assert y_agg_boundary_to_toggle___from_func == y_agg_boundary_to_toggle
#         assert equals(y_agg_el_ex_low___from_func[e, t], y_agg_el_ex_low[e, t])
#         assert equals(y_agg_el_ex_high___from_func[e, t], y_agg_el_ex_high[e, t])


# y_agg_boundary_el_ex_high - y_agg_boundary_el_ex_low



e: 0
t: 0

y_agg_el_above_boundary_VoT[e, t]: 213.94551690727567
demand_el_above_boundary_VoT: 213.94551690727567
y_agg_in_above_boundary_VoT[e, t]: 0.0
demand_in_above_boundary_VoT: 0

e: 0
t: 1

y_agg_el_above_boundary_VoT[e, t]: 421.4284925043978
demand_el_above_boundary_VoT: 421.4284925043978
y_agg_in_above_boundary_VoT[e, t]: 0.0
demand_in_above_boundary_VoT: 0

e: 0
t: 2

y_agg_el_above_boundary_VoT[e, t]: 297.5293864294237
demand_el_above_boundary_VoT: 297.5293864294237
y_agg_in_above_boundary_VoT[e, t]: 159.71778737860973
demand_in_above_boundary_VoT: 159.71778737860973

e: 0
t: 3

y_agg_el_above_boundary_VoT[e, t]: 213.94551690727567
demand_el_above_boundary_VoT: 213.94551690727567
y_agg_in_above_boundary_VoT[e, t]: 718.0701325843745
demand_in_above_boundary_VoT: 718.0701325843745

e: 0
t: 4

y_agg_el_above_boundary_VoT[e, t]: 115.15744798402402
demand_el_above_boundary_VoT: 115.15744798402402
y_agg_in_above_boundary_VoT[e, t]: 120.00603652178778
demand_in_above_boundary_VoT:

In [48]:
# y_agg_boundary_to_toggle

## Check if DBCP initialization outperforms CBCP

In [49]:
# def compute_obj_DBCP_together(y, x, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, num_edges, T):
#     obj = {}
    
#     lambda_E, lambda_R, lambda_I = lambda_list
    
#     for e in range(num_edges):
#         obj[(e, 'E')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + (1 - alpha[e, t]) * tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) )

#         obj[(e, 'R')] = sum( y[(od, g, e, 0, t)] * (1 - alpha[e, t]) * tau[e, t] \
#                             for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 0, t)] * tau[e, t] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[(e, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
#                         + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
#                             for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

#         obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
#     obj['total'] = sum([obj[e] for e in range(num_edges)])
    
#     return obj


def compute_obj_DBCP(y, x, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, num_edges, T):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    for e in range(num_edges):
        for t in range(T):
            
#             print("e:", e)
#             print("t:", t)
            
            obj[(e, t, 'E')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + (1 - alpha[e, t]) * tau[e, t]) \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                for od in edge_to_od_dict[e] for g in el_indices)
            
            obj[(e, t, 'R')] = sum( y[(od, g, e, 0, t)] * (1 - alpha[e, t]) * tau[e, t] \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] * tau[e, t] \
                                    for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t, 'I')] = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[(e, 0, t)] + tau[e, t]) \
                                for od in edge_to_od_dict[e] for g in in_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[(e, 1, t)] \
                                for od in edge_to_od_dict[e] for g in in_indices)

            obj[(e, t)] = lambda_E * obj[(e, t, 'E')] - lambda_R * obj[(e, t, 'R')] + lambda_I * obj[(e, t, 'I')]  
        
        obj[(e, 'E')] = sum([obj[(e, t, 'E')] for t in range(T)])
        obj[(e, 'R')] = sum([obj[(e, t, 'R')] for t in range(T)])
        obj[(e, 'I')] = sum([obj[(e, t, 'I')] for t in range(T)])
        
        obj[e] = lambda_E * obj[(e, 'E')] - lambda_R * obj[(e, 'R')] + lambda_I * obj[(e, 'I')]  
    
    obj['E'] = sum([obj[(e, 'E')] for e in range(num_edges)])
    obj['R'] = sum([obj[(e, 'R')] for e in range(num_edges)])
    obj['I'] = sum([obj[(e, 'I')] for e in range(num_edges)])
    
    obj['total'] = sum([obj[e] for e in range(num_edges)])
    
    return obj


def compute_obj_DBCP_at_e_t(y, travel_times, VoT_array, tau, alpha, edge_to_od_dict, lambda_list, e, t):
    obj = {}
    
    lambda_E, lambda_R, lambda_I = lambda_list
    
    obj_E = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[0] + (1 - alpha) * tau) \
                                for od in edge_to_od_dict[e] for g in el_indices) \
                            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[1] \
                                for od in edge_to_od_dict[e] for g in el_indices)
    
    obj_R = sum( y[(od, g, e, 0, t)] * (1 - alpha) * tau \
                        for od in edge_to_od_dict[e] for g in el_indices) \
                        + sum( y[(od, g, e, 0, t)] * tau \
                            for od in edge_to_od_dict[e] for g in in_indices)

    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times[0] + tau) \
                        for od in edge_to_od_dict[e] for g in in_indices) \
                    + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times[1] \
                        for od in edge_to_od_dict[e] for g in in_indices)

    obj = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

    return obj, obj_E, obj_R, obj_I


In [50]:
def extract_y_at_e_t(y, e, t):
    
    y_at_e_t = {}
    
    for key in y.keys():
        if key[2] == e and key[4] == t:
            y_at_e_t[key] = y[key]
    
    return y_at_e_t

In [51]:
# Compute aggregate flows and other characteristics

y_agg_DBCP_init, x_DBCP_init \
    = compute_y_agg_x(y = y_DBCP_init, \
                      edge_to_od_dict = edge_to_od_dict,\
                      num_edges = num_edges, \
                      T = T)

travel_times_DBCP_init, avg_travel_times_DBCP_init \
    = compute_travel_times(y_agg = y_agg_DBCP_init, \
                           x = x_DBCP_init, \
                           edge_to_od_dict = edge_to_od_dict,\
                           coeff_input = coeff_input, \
                           num_gp_lanes = num_gp_lanes,\
                           num_edges = num_edges, \
                           T = T)

percent_on_express_DBCP_init \
    = compute_percent_on_express(y_agg = y_agg_DBCP_init,\
                                 x = x_DBCP_init, \
                                 num_edges = num_edges, \
                                 T = T)

lambda_list = [lambda_E, lambda_R, lambda_I]

# obj_DBCP_init_together \
#     = compute_obj_DBCP_together(y = y_DBCP_init, \
#                        x = x_DBCP_init, \
#                        travel_times = travel_times_DBCP_init, \
#                        VoT_array = VoT_array, \
#                        tau = argmin_tau, \
#                        alpha = init_alpha, \
#                        edge_to_od_dict = edge_to_od_dict, \
#                        lambda_list = lambda_list, \
#                        num_edges = num_edges, \
#                        T = T)

# obj_DBCP_init \
#     = compute_obj_DBCP(y = y_DBCP_init, \
#                        x = x_DBCP_init, \
#                        travel_times = travel_times_DBCP_init, \
#                        VoT_array = VoT_array, \
#                        tau = argmin_tau, \
#                        alpha = init_alpha, \
#                        edge_to_od_dict = edge_to_od_dict, \
#                        lambda_list = lambda_list, \
#                        num_edges = num_edges, \
#                        T = T)

# print("travel_times_DBCP_init:", travel_times_DBCP_init)

obj_DBCP_init = {}
for e in range(num_edges):
    for t in range(T):
        
        travel_times_DBCP_init_at_e_t = [travel_times_DBCP_init[(e, k, t)] for k in [0, 1]]

        obj_DBCP_init[(e, t)], obj_DBCP_init[(e, t, 'E')], \
                obj_DBCP_init[(e, t, 'R')], obj_DBCP_init[(e, t, 'I')] \
            = compute_obj_DBCP_at_e_t(y = y_DBCP_init, \
                                      travel_times = travel_times_DBCP_init_at_e_t, \
                                      VoT_array = VoT_array, \
                                      tau = argmin_tau[e, t], \
                                      alpha = init_alpha[e, t], \
                                      edge_to_od_dict = edge_to_od_dict, \
                                      lambda_list = lambda_list, \
                                      e = e, \
                                      t = t)

    obj_DBCP_init[(e, 'E')] = sum([obj_DBCP_init[(e, t, 'E')] for t in range(T)])
    obj_DBCP_init[(e, 'R')] = sum([obj_DBCP_init[(e, t, 'R')] for t in range(T)])
    obj_DBCP_init[(e, 'I')] = sum([obj_DBCP_init[(e, t, 'I')] for t in range(T)])

    obj_DBCP_init[e] = lambda_E * obj_DBCP_init[(e, 'E')] - lambda_R * obj_DBCP_init[(e, 'R')] + lambda_I * obj_DBCP_init[(e, 'I')]  

obj_DBCP_init['total'] = sum([obj_DBCP_init[e] for e in range(num_edges)])


# print("percent_on_express_DBCP_init:\n", percent_on_express_DBCP_init)

# obj_DBCP_init['total']


e: 0
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 12411.931193933562
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 7538.801148524434

e: 1
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 22302.47356395012
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 8834.40910602634

e: 2
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 70843.47513214104
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 11490.113280262252

e: 3
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 17607.3440613401
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 9951.409408082503

e: 4
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 96899.19216345657
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 13178.963445514572

e: 5
sum

In [52]:
# print(obj_DBCP_init == obj___from_func)

print("obj_CBCP_opt[total]:", obj_CBCP_opt['total'])
print("obj_DBCP_init[total]:", obj_DBCP_init['total'])
# obj_DBCP_init['total']

obj_CBCP_opt[total]: 794113.873307937
obj_DBCP_init[total]: 793039.4996520553


In [53]:
# for k in np.linspace(0, 10, 5):
#     print(k)

In [54]:
# y_DBCP_init_at_e_t, VoT_in_boundary[e, t], VoT_el_boundary[e, t], init_alpha[e, t] \
#             = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'in')], \
#                                      y_agg_el_at_e_t = y_agg_CBCP_opt[(e, 0, t, 'el')], \
#                                      VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
#                                      VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
#                                      e = e, \
#                                      t = t)

In [55]:
## Toggle code

y_DBCP_init_min = {}

obj_DBCP_init_min = np.zeros((num_edges, T))

num_y_toggle = 101

for e in range(num_edges):
    for t in range(T):
        
        print()
        print("e:", e)
        print("t:", t)
        print()
        
        # Compute obj (DBCP) for DBCP init, as baseline:
        
        travel_times_DBCP_init_at_e_t = [travel_times_DBCP_init[(e, k, t)] for k in [0, 1]]
        
        y_DBCP_init_min_at_e_t = extract_y_at_e_t(y_DBCP_init, e, t)
        
        obj_DBCP_init_min_at_e_t, _, _, _ \
            = compute_obj_DBCP_at_e_t(y = y_DBCP_init, \
                                      travel_times = travel_times_DBCP_init_at_e_t, \
                                      VoT_array = VoT_array, \
                                      tau = argmin_tau[e, t], \
                                      alpha = init_alpha[e, t], \
                                      edge_to_od_dict = edge_to_od_dict, \
                                      lambda_list = lambda_list, \
                                      e = e, \
                                      t = t)
        
        if y_agg_boundary_to_toggle[(e, t)]:
            y_agg_el_ex_arr = np.linspace(y_agg_el_ex_low[e, t], y_agg_el_ex_high[e, t], num_y_toggle)
            
            for y_agg_el_ex in y_agg_el_ex_arr:
                
                y_agg_in_ex = x_DBCP_init[(e, 0, t)] - y_agg_el_ex
                
#                 print("y_agg_el_ex:", y_agg_el_ex)
#                 print("y_agg_in_ex:", y_agg_in_ex)
                
                y_DBCP_toggle_at_e_t, _, _, _ \
                    = y_agg_to_y_DBCP_at_e_t(y_agg_in_at_e_t = y_agg_in_ex, \
                                             y_agg_el_at_e_t = y_agg_el_ex, \
                                             VoTs_demands_annotated_in = dict_VoTs_demands_annotated[(e, t, "in")], \
                                             VoTs_demands_annotated_el = dict_VoTs_demands_annotated[(e, t, "el")], \
                                             e = e, \
                                             t = t)

                obj_toggle_at_e_t, _, _, _ \
                    = compute_obj_DBCP_at_e_t(y = y_DBCP_toggle_at_e_t, \
                                              travel_times = travel_times_DBCP_init_at_e_t, \
                                              VoT_array = VoT_array, \
                                              tau = argmin_tau[e, t], \
                                              alpha = init_alpha[e, t], \
                                              edge_to_od_dict = edge_to_od_dict, \
                                              lambda_list = lambda_list, \
                                              e = e, \
                                              t = t)
                
                if obj_toggle_at_e_t < obj_DBCP_init_min_at_e_t:
                    
                    y_DBCP_init_min_at_e_t = y_DBCP_toggle_at_e_t
                    obj_DBCP_init_min_at_e_t = obj_toggle_at_e_t
        
        y_DBCP_init_min = {**y_DBCP_init_min, **y_DBCP_init_min_at_e_t}
        obj_DBCP_init_min[e, t] = obj_DBCP_init_min_at_e_t


e: 0
t: 0


e: 0
t: 1


e: 0
t: 2


e: 0
t: 3


e: 0
t: 4


e: 1
t: 0


e: 1
t: 1


e: 1
t: 2


e: 1
t: 3


e: 1
t: 4


e: 2
t: 0


e: 2
t: 1


e: 2
t: 2


e: 2
t: 3


e: 2
t: 4


e: 3
t: 0


e: 3
t: 1


e: 3
t: 2


e: 3
t: 3


e: 3
t: 4


e: 4
t: 0


e: 4
t: 1


e: 4
t: 2


e: 4
t: 3


e: 4
t: 4


e: 5
t: 0


e: 5
t: 1


e: 5
t: 2


e: 5
t: 3


e: 5
t: 4


e: 6
t: 0


e: 6
t: 1


e: 6
t: 2


e: 6
t: 3


e: 6
t: 4



In [56]:
# y_DBCP_init_min

print("obj_CBCP_opt[total]:\n", obj_CBCP_opt['total'])
print()
print("obj_DBCP_init[total]:\n", obj_DBCP_init['total'])
print()
print("np.sum(obj_DBCP_init_min):\n", np.sum(obj_DBCP_init_min))
print()

# np.sum(obj_DBCP_init_min)
# obj_CBCP_opt['total'] - np.sum(obj_DBCP_init_min)
print("Improvement fraction:\n", (obj_CBCP_opt['total'] - np.sum(obj_DBCP_init_min)) / obj_CBCP_opt['total'] * 100, "%")

obj_CBCP_opt[total]:
 794113.873307937

obj_DBCP_init[total]:
 793039.4996520553

np.sum(obj_DBCP_init_min):
 791563.0632921016

Improvement fraction:
 0.3212146395591695 %


In [57]:
# y_DBCP_init_min

# compute_obj_DBCP(y_DBCP_init_min)

# obj_DBCP_init_min = compute_obj_DBCP_at_e_t(y = y_DBCP_init_min, \
#                                               travel_times = travel_times_DBCP_init_at_e_t, \
#                                               VoT_array = VoT_array, \
#                                               tau = argmin_tau[e, t], \
#                                               alpha = init_alpha[e, t], \
#                                               edge_to_od_dict = edge_to_od_dict, \
#                                               lambda_list = lambda_list, \
#                                               e = e, \
#                                               t = t)

y_agg_DBCP_init_min, x_DBCP_init_min = compute_y_agg_x(y = y_DBCP_init_min, \
                                                       edge_to_od_dict = edge_to_od_dict, \
                                                       num_edges = num_edges, \
                                                       T = T)

travel_times_DBCP_init_min, _ = compute_travel_times(y_agg = y_agg_DBCP_init_min, \
                                                     x = x_DBCP_init_min, \
                                                     edge_to_od_dict = edge_to_od_dict, \
                                                     coeff_input = coeff_input, \
                                                     num_gp_lanes = num_gp_lanes, \
                                                     num_edges = num_edges, \
                                                     T = T)

obj_DBCP_init_min = compute_obj_DBCP(y = y_DBCP_init_min, \
                                     x = x_DBCP_init_min, \
                                     travel_times = travel_times_DBCP_init_min, \
                                     VoT_array = VoT_array, \
                                     tau = argmin_tau, \
                                     alpha = init_alpha, \
                                     edge_to_od_dict = edge_to_od_dict, \
                                     lambda_list = lambda_list, \
                                     num_edges = num_edges, \
                                     T = T)

obj_DBCP_init_min


e: 0
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 12465.175997058433
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 7538.801148524435

e: 1
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 22341.322950377515
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 8834.40910602634

e: 2
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 70861.84649521533
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 11490.113280262252

e: 3
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 17930.517099621273
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 9951.409408082503

e: 4
sum(y_agg[(e, k, t, 'el')] * travel_times[(e, k, t)] for k in range(2) for t in range(T)): 97028.96201214027
sum(y_agg[(e, k, t, 'el')] for k in range(2) for t in range(T)): 13178.963445514573

e: 5


{(0, 0, 'E'): 509.11586845828407,
 (0, 0, 'R'): 396.15760169350125,
 (0, 0, 'I'): 8485.758050460025,
 (0, 0): 8598.716317224807,
 (0, 1, 'E'): 464.2003020465236,
 (0, 1, 'R'): 168.2598903711484,
 (0, 1, 'I'): 7635.602947805441,
 (0, 1): 7931.5433594808155,
 (0, 2, 'E'): 465.9984041131797,
 (0, 2, 'R'): 337.3559214936678,
 (0, 2, 'I'): 7804.986236556809,
 (0, 2): 7933.62871917632,
 (0, 3, 'E'): 436.5069529821651,
 (0, 3, 'R'): 385.83607512756396,
 (0, 3, 'I'): 7986.574713235096,
 (0, 3): 8037.245591089697,
 (0, 4, 'E'): 529.9806072107126,
 (0, 4, 'R'): 381.95931340731016,
 (0, 4, 'I'): 8892.993871377703,
 (0, 4): 9041.015165181105,
 (0, 'E'): 2405.802134810865,
 (0, 'R'): 1669.5688020931916,
 (0, 'I'): 40805.915819435075,
 0: 41542.14915215275,
 (1, 0, 'E'): 933.5008879276738,
 (1, 0, 'R'): 28.972906816029813,
 (1, 0, 'I'): 12326.173807620942,
 (1, 0): 13230.701788732586,
 (1, 1, 'E'): 798.4265125801081,
 (1, 1, 'R'): 192.8109807072645,
 (1, 1, 'I'): 10990.668327002662,
 (1, 1): 11596.2

##  <font color='red'> (August 13) Compute (1) percentage using express lanes (overall, eligible, ineligible) and (2) Average travel times (express, general purpose). </font> 

In [58]:
argmin_tau

array([[0.92, 0.28, 0.46, 0.48, 0.94],
       [0.95, 0.32, 0.8 , 0.58, 0.7 ],
       [0.51, 2.23, 2.08, 1.35, 2.19],
       [1.27, 0.86, 1.72, 1.38, 2.47],
       [2.13, 2.29, 2.25, 0.58, 1.31],
       [0.53, 0.44, 0.69, 0.5 , 0.38],
       [0.79, 0.26, 0.47, 0.36, 0.75]])

##  <font color='red'> (August 13) Compare statistics against those w.r.t. the optimal CBCP policy. </font> 

## Computing lower bound to societal cost difference

In [59]:
## Computing lower bound to societal cost difference,
# i.e., the societal cost should be lower by at least this much: 

# obj_CBCP_opt

rev_increase = np.zeros((num_edges, T))
y_el_ex_CBCP_opt = np.zeros((num_edges, T))
rev_increase_total = 0.0

for e in range(num_edges):
    for t in range(T):
        y_el_ex_CBCP_opt[e, t] = sum([y_CBCP_opt[(od, g, e, 0, t)] \
                                      for od in edge_to_od_dict[e] for g in el_indices ])
        
        rev_increase[e, t] = (1 - init_alpha[e, t]) * argmin_tau[e, t] * y_el_ex_CBCP_opt[e, t]
        
        rev_increase_total += rev_increase[e, t]

# y_CBCP_opt

# y_el_ex_CBCP_opt

# rev_increase

# rev_increase_total

In [60]:
# np.sum(argmin_tau)
# argmin_tau
# y_CBCP_opt

obj_CBCP_opt

{(0, 0, 'E'): 495.1906551849078,
 (0, 0, 'R'): 361.3193571562317,
 (0, 0, 'I'): 8485.747633397628,
 (0, 0): 8619.618931426305,
 (0, 1, 'E'): 454.18973611145964,
 (0, 1, 'R'): 149.44334102065687,
 (0, 1, 'I'): 7635.60314340771,
 (0, 1): 7940.3495384985135,
 (0, 2, 'E'): 456.0534985435468,
 (0, 2, 'R'): 318.9823872563723,
 (0, 2, 'I'): 7804.986432836546,
 (0, 2): 7942.05754412372,
 (0, 3, 'E'): 428.1475657556268,
 (0, 3, 'R'): 367.8536012314232,
 (0, 3, 'I'): 7986.591979407653,
 (0, 3): 8046.885943931856,
 (0, 4, 'E'): 509.5178745855066,
 (0, 4, 'R'): 266.67835247210246,
 (0, 4, 'I'): 8893.417178784477,
 (0, 4): 9136.25670089788,
 (0, 'E'): 2343.0993301810477,
 (0, 'I'): 40806.34636783401,
 (0, 'R'): 1464.2770391367865,
 0: 41685.168658878276,
 (1, 0, 'E'): 931.3762335853235,
 (1, 0, 'R'): 0.0015081260626418772,
 (1, 0, 'I'): 12324.34850817442,
 (1, 0): 13255.723233633682,
 (1, 1, 'E'): 786.5458496090615,
 (1, 1, 'R'): 170.7924963753395,
 (1, 1, 'I'): 10990.668608264601,
 (1, 1): 11606.4

# DBCP Optimization

In [61]:
def proj_tau_alpha(T, num_edges, tau, alpha, tau_max):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    alpha_feas = np.clip(alpha, 0.0, 1.0)

    print()
    print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print()
    print("alpha_feas:", alpha_feas)
    print()
    
#     print("alpha:", alpha)
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=3), \
            np.round(np.maximum(alpha_feas, 0.0), decimals=3)
    
#     return tau_feas.value, alpha_feas



def proj_tau_alpha_at_e_t(T, num_edges, tau, alpha, tau_max):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

#     assert tau == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
    constraints += [tau_feas <= tau_max]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()
    
    alpha_feas = np.clip(alpha, 0.0, 1.0)

    print()
    print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print()
    print("alpha_feas:", alpha_feas)
    print()
    
    print("np.round(np.maximum(tau_feas.value, 0.0), decimals=3):", \
          np.round(np.maximum(tau_feas.value, 0.0), decimals=3))
    print("np.round(np.maximum(alpha_feas, 0.0), decimals=3):", \
         np.round(np.maximum(alpha_feas, 0.0), decimals=3))
    
#     print("alpha:", alpha)
#     print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=3), \
            np.round(np.maximum(alpha_feas, 0.0), decimals=3)
    
#     return tau_feas.value, alpha_feas



In [62]:
# fl = np.array([1])
# type(fl)

In [68]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_DBCP_at_e_t(num_gp_lanes, tau, alpha, od_to_edges_list_full, \
                      demand_array, VoT_array, el_indices, in_indices, \
                      coeff_input, e, t):
    
    arr_indicator = np.zeros((num_edges, T))
    arr_indicator[e, t] = 1.0
    
#     print("type(tau):", type(tau))
#     print("type(alpha):", type(alpha))
#     print("tau.shape:", tau.shape)
#     print("alpha:", alpha.shape)
    
    if type(tau) == float or type(tau) == int or tau.shape == ():
#         print("Modifying tau shape")
        tau = tau * arr_indicator
    if type(alpha) == float or type(alpha) == int or alpha.shape == ():
#         print("Modifying alpha shape")
        alpha = alpha * arr_indicator
    
#     print("arr_indicator:\n", arr_indicator)
#     print("tau:", tau)
#     print("alpha:", alpha)
    
    assert tau.shape == alpha.shape, "We must have tau.shape == alpha.shape!"
    assert np.all((alpha >= -1E-3) & (alpha <= 1.001))
    
    alpha = np.clip(alpha, 0.0, 1.0)
    
#     print("tau (in solve_DBCP_direct):", tau)
    
#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    coeff = coeff_input
    
    print("coeff:\n", coeff)
    print()
    
#     assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T."
    assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
#     assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
#                                             demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, g, e, k, t) = (od, income group, edge, "lane", time)
    
#     od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
#                           for od in range(od_to_edges_array.shape[0])]
    
#     edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#                    for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for g in el_indices:
#             for k in [0, 1, 2]:
            for k in [0, 1]:
                y[(od, g, e, k, t)] = cp.Variable(1)
        for g in in_indices:
            for k in [0, 1]:
                y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for k in [0, 1]:
        x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    func += coeff[0, e] * x[(e, 0, t)] \
            + 0.5 * coeff[1, e] * cp.square(cp.maximum(x[(e, 0, t)] - coeff[2, e], 0))
    func += coeff[0, e] * x[(e, 1, t)] \
            + 0.5 * coeff[1, e] * num_gp_lanes * cp.square(cp.maximum(x[(e, 1, t)]/num_gp_lanes - coeff[2, e], 0))
    
#     print("VoT_array.shape:", VoT_array.shape)
#     print("el_indices:", el_indices)
#     print("in_indices:", in_indices)

    
    for od in range(num_od):
        for g in el_indices:
            
#             print("alpha[e, t]:", alpha[e, t])
#             print("tau[e, t]:", tau[e, t])
#             print("VoT_array[od, g, t]:", VoT_array[od, g, t])
            
            func += (1 - alpha[e, t]) * tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]
#             func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g, t]
#         for g in in_indices:
#             func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g, t]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
#     constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) for g in el_indices for k in [0, 1, 2]]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) for g in in_indices + el_indices for k in [0, 1]]
    constraints += [x[(e, k, t)] >= 0.0 for k in [0, 1]]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("alpha:", alpha)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []
    
    ## Edge contraints:
    constraints += [sum( y[(od, g, e, k, t)] for od in edge_to_ods[e] for g in el_indices + in_indices) \
                        == x[(e, k, t)] for k in [0, 1] ]

    ## Group flow constraints:
    for od in edge_to_ods[e]:
        for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
            assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
            od_g_e_t_list += [(od, g, e, t)]

#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]

#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]

#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

#     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
#                     for od in edge_to_ods[e] for g in el_indices]
    constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                    for od in edge_to_ods[e] for g in el_indices]
    constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                    for od in edge_to_ods[e] for g in in_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for od in edge_to_ods[e]:
        for g in el_indices:
#             for k in [0, 1, 2]:
            for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
        for g in in_indices:
            for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


In [73]:
def welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha, \
                       demand_array, VoT_array, num_el, od_to_edges_array, y, \
                       coeff_input, e, t):

#     coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)

    assert np.all((alpha >= -1E-3) & (alpha <= 1.001)), "We must have alpha in [0, 1] component-wise!"
    
    coeff = coeff_input
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                           if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

#     assert len(tau.shape) == 2, "tau should be 2-dimensional."
#     assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
#     assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros(2)
    x[0] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] for g in el_indices + in_indices)
    x[1] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] for g in el_indices + in_indices)

        
#     print()
#     print("in_indices:", in_indices)
#     print("el_indices:", el_indices)
#     print()
    
    ## Compute lane latencies:
    
    ell = np.zeros(2)
    ell[0] = coeff[0, e] + coeff[1, e] * max(x[0] - coeff[2, e], 0)
    ell[1] = coeff[0, e] + coeff[1, e] * max(x[1]/num_gp_lanes - coeff[2, e], 0)    

    # Computing obj_E, obj_I, obj_R, and welfare at edge e and time t:
    
    obj_E = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[0] + (1 - alpha) * tau ) \
                for od in edge_to_od_dict[e] for g in el_indices) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[1] \
                  for od in edge_to_od_dict[e] for g in el_indices)
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g, t] * ell[0] + tau) \
                for od in edge_to_od_dict[e] for g in in_indices) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g, t] * ell[1] \
                for od in edge_to_od_dict[e] for g in in_indices)
    obj_R = sum( y[(od, g, e, 0, t)] * (1 - alpha) * tau \
                for od in edge_to_od_dict[e] for g in el_indices) \
            + sum( y[(od, g, e, 0, t)] * tau \
                for od in edge_to_od_dict[e] for g in in_indices)

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("In the welfare_obj_at_e_t function, -1:")
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


## Running zeroth-order gradient descent

In [74]:
T = 5

# num_iters = 1000
# num_iters = 100
num_iters = 200
# num_iters = 3

# num_el = 3
num_groups = demand_array.shape[1]

assert VoT_array.shape[2] == T, "We should have VoT_array.shape[2] == T"
assert demand_array.shape == VoT_array[:, :, 0].shape, "demand_array and VoT_array[:, :, 0] should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
# num_edges = 6
num_edges = 7
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

error_bound = 1E-2
diffs_num_cols = 5

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


## coeff_input: const, slope, x-coordinate of transition point

# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))


coeff_input = np.zeros((3, num_edges))
for counter, city in enumerate(dict_latency_params.keys()):
    coeff_input[0, counter] = dict_latency_params[city]["Latency (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Flow (at bend)"]


# def solve_DBCP_direct(T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, alpha, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

el_indices = list(range(num_el))
in_indices = list(range(num_el, num_groups))

tau_max_from_latency = np.zeros((num_edges, T))
VoT_max_el = np.zeros((num_edges, T))
for e in range(num_edges):
    for t in range(T):
        VoT_max_el[e, t] = max([VoT_array[od_index, group_index, t] for od_index in edge_to_ods[e] \
                                for group_index in el_indices])
        tau_max_from_latency[e, t] = VoT_max_el[e, t] * (latency_max(demand_edges_array[e], coeff_input[:, e]) - coeff_input[0, e]) \

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)

print("tau_upper_limit:\n", tau_upper_limit)

d = 2

# fraction_tau_max = 0.5

# tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit * fraction_tau_max, tau_upper_limit)
# print()
# print("tau[:, :, 0]:", tau[:, :, 0])
# print()
# print("np.sum(tau[:, :, 0]):", np.sum(tau[:, :, 0]))

# od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
#                          for od in range(od_to_edges_array.shape[0])]


# # To disable when restarting from scratch
# tau[:, :, 0] = tau_next_init
# alpha[:, :, 0] = alpha_next_init
# eta[0] = eta_bar * (index_next_init+1)**(-1/2) * d**(-1)
# delta[0] = delta_bar * (index_next_init+1)**(-1/4) * d**(-1/2)

tau_upper_limit:
 [[1.02950495 1.01636822 0.98927591 0.93381404 0.99927582]
 [0.80480983 0.7630661  0.75883561 0.73000399 0.74590899]
 [3.05095521 2.77988065 2.87667145 2.76737359 2.82766788]
 [2.98669068 2.8592147  2.95876778 2.8561173  2.90836571]
 [4.66065813 4.66853574 4.61708512 4.46823839 4.44841466]
 [0.56349006 0.53467278 0.53734074 0.53409357 0.53307654]
 [0.63847292 0.60582097 0.61773543 0.60516468 0.60401232]]


In [75]:
print("argmin_tau:\n", argmin_tau)
print()
print("init_alpha:\n", init_alpha)

argmin_tau:
 [[0.92 0.28 0.46 0.48 0.94]
 [0.95 0.32 0.8  0.58 0.7 ]
 [0.51 2.23 2.08 1.35 2.19]
 [1.27 0.86 1.72 1.38 2.47]
 [2.13 2.29 2.25 0.58 1.31]
 [0.53 0.44 0.69 0.5  0.38]
 [0.79 0.26 0.47 0.36 0.75]]

init_alpha:
 [[0.85596331 0.87689407 0.86854752 0.85819808 0.85440341]
 [0.8454951  0.87720412 0.85076804 0.85581165 0.8516927 ]
 [0.86840947 0.87852273 0.86854752 0.88331776 0.8627252 ]
 [0.85557171 0.9449055  0.88818808 0.88331776 0.85050779]
 [0.87957513 0.88477124 0.88944531 0.88519769 0.94608763]
 [0.85596331 0.88477124 0.85351709 0.88543049 0.8807204 ]
 [0.85596331 0.88702193 0.86830451 0.88841123 0.87089765]]


In [76]:
time_1 = time.time()


## Set lambdas:
# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0

## Check lambda:
print("lambda_E:", lambda_E)
print("lambda_R:", lambda_R)
print("lambda_I:", lambda_I)
print()

## Initialize tau, alpha values:

# filename_segment = str(int(lambda_E)) + '_' + str(int(lambda_R)) + '_' + str(int(lambda_I))
# # filename_segment = '4654283787'

# directory_inits = '../data/opt_tolls_subsidies_metrics/'
# df_inits = pd.read_csv(directory_inits + 'inits___' + filename_segment + '.csv')

# print("filename_segment:", filename_segment)

welfare_min_array = np.zeros((num_edges, T))
obj_E_min_at_e_t = np.zeros((num_edges, T))
obj_R_min_at_e_t = np.zeros((num_edges, T))
obj_I_min_at_e_t = np.zeros((num_edges, T))

delta = np.zeros(num_iters)
eta = np.zeros(num_iters)

# eta_bar = 1E-5
# delta_bar = 1E-5

eta_bar = 0.1
delta_bar = 0.1

argmin_tau_new = np.zeros((num_edges, T))
argmin_alpha_new = np.zeros((num_edges, T))

y_opt_DBCP_new = copy.deepcopy(y_DBCP_init_min)

for e in range(num_edges):
    for t in range(T):
        
        print("e:", e)
        print("t:", t)

# for e in [0]:
#     for t in [0]:    
        
        tau_e_t = np.zeros(num_iters)
        tau_perturbed_e_t = np.zeros(num_iters)

        alpha_e_t = np.zeros(num_iters)
        alpha_perturbed_e_t = np.zeros(num_iters)
        
        tau_e_t[0] = argmin_tau[e, t]
        alpha_e_t[0] = init_alpha[e, t]
        
        welfare_min_array[e, t] = obj_DBCP_init_min[(e, t)]
        obj_E_min_at_e_t[e, t] = obj_DBCP_init_min[(e, t, 'E')]
        obj_R_min_at_e_t[e, t] = obj_DBCP_init_min[(e, t, 'R')]
        obj_I_min_at_e_t[e, t] = obj_DBCP_init_min[(e, t, 'I')]
        argmin_tau_new[e, t] = argmin_tau[e, t]
        argmin_alpha_new[e, t] = init_alpha[e, t]
        
        for i in range(num_iters-1):
        
            time_1_at_e_t = time.time()

            print()
            print("Iter:", i)

            eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
            delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
            w_i_unnormalized = np.random.randn(2)
            w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
            
        #     print("w_i:", w_i)
            tau_perturbed_e_t[i] = tau_e_t[i] + delta[i] * w_i[0]
            alpha_perturbed_e_t[i] = alpha_e_t[i] + delta[i] * w_i[1]

        #     if tau_perturbed[e, t, i] < alpha_perturbed[e, t, i] or tau_perturbed[e, t, i] < 0 alpha_perturbed[e, t, i] < 0:

            print("i:\n", i)
            print("tau_e_t[i], before projection:\n", tau_e_t[i])
            print("alpha_e_t[i], before projection:\n", alpha_e_t[i])
            print("tau_perturbed_e_t[i], before projection:\n", tau_perturbed_e_t[i])
            print("alpha_perturbed_e_t[i], before projection:\n", alpha_perturbed_e_t[i])
            
            print("tau_perturbed_e_t[i]:", tau_perturbed_e_t[i])
            print("alpha_perturbed_e_t[i]:", alpha_perturbed_e_t[i])
            print("tau_upper_limit:", tau_upper_limit)

            tau_perturbed_e_t[i], alpha_perturbed_e_t[i] \
                = proj_tau_alpha_at_e_t(T, num_edges, tau_perturbed_e_t[i], alpha_perturbed_e_t[i], \
                                        tau_max = tau_upper_limit[e, t])

            print("tau[e, t, i], after projection:\n", tau_e_t[i])
            print("alpha[e, t, i], after projection:\n", alpha_e_t[i])
            print("tau_perturbed[e, t, i], after projection:\n", tau_perturbed_e_t[i])
            print("alpha_perturbed[e, t, i], after projection:\n", alpha_perturbed_e_t[i])
            
#             def solve_DBCP_at_e_t(num_gp_lanes, tau, alpha, od_to_edges_list_full, \
#                       demand_array, VoT_array, el_indices, in_indices, \
#                       coeff_input, e, t):

            print()
#             print("demand_array_temp:", demand_array_temp)
            print("VoT_array.shape:", VoT_array.shape)
            print()

            y_values_at_e_t = \
                solve_DBCP_at_e_t(num_gp_lanes, tau_e_t[i], alpha_e_t[i], od_to_edges_list_full, \
                                         demand_array_temp, VoT_array, el_indices, in_indices, \
                                         coeff_input, e, t)

            y_perturbed_values_at_e_t = \
                solve_DBCP_at_e_t(num_gp_lanes, tau_perturbed_e_t[i], alpha_perturbed_e_t[i], od_to_edges_list_full, \
                                                   demand_array_temp, VoT_array, el_indices, in_indices, \
                                                   coeff_input, e, t)
            
#             def welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, alpha, \
#                        demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                        coeff_input, e, t):

            welfare_at_e_t, obj_E_at_e_t, obj_R_at_e_t, obj_I_at_e_t = \
                welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                                       tau_e_t[i], alpha_e_t[i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                                       y = y_values_at_e_t, coeff_input = coeff_input, e = e, t = t)
            
            welfare_perturbed_at_e_t, obj_E_perturbed_at_e_t, obj_R_perturbed_at_e_t, obj_I_perturbed_at_e_t \
                = welfare_obj_at_e_t(num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                              tau_perturbed_e_t[i], alpha_perturbed_e_t[i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                              y = y_perturbed_values_at_e_t, coeff_input = coeff_input, e = e, t = t)
            
            print()
            print("welfare_at_e_t:", welfare_at_e_t)
            print("obj_E_at_e_t:", obj_E_at_e_t)
            print("obj_R_at_e_t:", obj_R_at_e_t)
            print("obj_I_at_e_t:", obj_I_at_e_t)
            print()
            print("welfare_perturbed_at_e_t:", welfare_perturbed_at_e_t)
            print("obj_E_perturbed_at_e_t:", obj_E_perturbed_at_e_t)
            print("obj_R_perturbed_at_e_t:", obj_R_perturbed_at_e_t)
            print("obj_I_perturbed_at_e_t:", obj_I_perturbed_at_e_t)
            print()
            
            if welfare_at_e_t < welfare_min_array[e, t]:
                
                welfare_min_array[e, t] = welfare_at_e_t
                obj_E_min_at_e_t[e, t] = obj_E_at_e_t
                obj_R_min_at_e_t[e, t] = obj_R_at_e_t
                obj_I_min_at_e_t[e, t] = obj_I_at_e_t
                argmin_tau_new[e, t] = tau_e_t[i]
                argmin_alpha_new[e, t] = alpha_e_t[i]
                
                for key in list(y_values_at_e_t.keys()):
                    y_opt_DBCP_new[key] = y_values_at_e_t[key]
            
#             print("quantity:", tau[e, t, i] - eta[i] * (d/delta[i]) * w_i[0] \
#                                 * (welfare_perturbed_at_e_t - welfare_at_e_t))

            tau_e_t[i+1] = tau_e_t[i] - eta[i] * (d/delta[i]) * w_i[0] \
                                * (welfare_perturbed_at_e_t - welfare_at_e_t)

            alpha_e_t[i+1] = alpha_e_t[i] - eta[i] * (d/delta[i]) * w_i[1] \
                                * (welfare_perturbed_at_e_t - welfare_at_e_t)

            print("tau_e_t[:, :, i+1], before projection:\n", tau_e_t[i+1])
            print("alpha_e_t[:, :, i+1], before projection:\n", alpha_e_t[i+1])
            
            tau_e_t[i+1], alpha_e_t[i+1] \
                = proj_tau_alpha_at_e_t(T, num_edges, tau_e_t[i+1], alpha_e_t[i+1], \
                                        tau_max = tau_upper_limit)

            print("tau_e_t[:, :, i+1], after projection:\n", tau_e_t[i+1])
            print("alpha_e_t[:, :, i+1], after projection:\n", alpha_e_t[i+1])
        
            if i >= diffs_num_cols + 2:
                tau_diffs = np.max(np.absolute(tau_e_t[i - diffs_num_cols : i-1] - tau_e_t[i - diffs_num_cols+1 : i]))
                alpha_diffs = np.max(np.absolute(alpha_e_t[i-diffs_num_cols : i-1] - alpha_e_t[i-diffs_num_cols+1 : i]))

        #         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
        #         print("alpha[:, :, 0:10]:", alpha[:, :, 0:10])
                print("tau_diffs:\n", tau_diffs)
                print("alpha_diffs:\n", alpha_diffs)
                print()

                if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(alpha_diffs))) < error_bound:
                    print("Within error bound.")
                    break

            time_2_at_e_t = time.time()

            
            print()
            print("Iteration count:", i)
            print("Time:", time_2_at_e_t - time_1_at_e_t)
            print()

time_2 = time.time()
print("Time:", time_2 - time_1)


lambda_E: 1.0
lambda_R: 1.0
lambda_I: 1.0

e: 0
t: 0

Iter: 0
i:
 0
tau_e_t[i], before projection:
 0.92
alpha_e_t[i], before projection:
 0.8559633127649717
tau_perturbed_e_t[i], before projection:
 0.9607151156152931
alpha_perturbed_e_t[i], before projection:
 0.9137757626389679
tau_perturbed_e_t[i]: 0.9607151156152931
alpha_perturbed_e_t[i]: 0.9137757626389679
tau_upper_limit: [[1.02950495 1.01636822 0.98927591 0.93381404 0.99927582]
 [0.80480983 0.7630661  0.75883561 0.73000399 0.74590899]
 [3.05095521 2.77988065 2.87667145 2.76737359 2.82766788]
 [2.98669068 2.8592147  2.95876778 2.8561173  2.90836571]
 [4.66065813 4.66853574 4.61708512 4.46823839 4.44841466]
 [0.56349006 0.53467278 0.53734074 0.53409357 0.53307654]
 [0.63847292 0.60582097 0.61773543 0.60516468 0.60401232]]

tau_feas.value: [0.961]

alpha_feas: 0.9137757626389679

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.961]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.914
tau[e, t, i], after projection:



prob.status: optimal

welfare_at_e_t: 8196.835787222684
obj_E_at_e_t: 465.70852109872885
obj_R_at_e_t: 637.512259383189
obj_I_at_e_t: 8368.639525507144

welfare_perturbed_at_e_t: 8196.83444562841
obj_E_perturbed_at_e_t: 465.70867351714816
obj_R_perturbed_at_e_t: 688.4151151653058
obj_I_perturbed_at_e_t: 8419.540887276567

tau_e_t[:, :, i+1], before projection:
 0.5271209584577936
alpha_e_t[:, :, i+1], before projection:
 0.8392628925913544

tau_feas.value: [0.527]

alpha_feas: 0.8392628925913544

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.527]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.839
tau_e_t[:, :, i+1], after projection:
 0.527
alpha_e_t[:, :, i+1], after projection:
 0.839

Iteration count: 3
Time: 0.3084869384765625


Iter: 4
i:
 4
tau_e_t[i], before projection:
 0.527
alpha_e_t[i], before projection:
 0.839
tau_perturbed_e_t[i], before projection:
 0.5657455101125944
alpha_perturbed_e_t[i], before projection:
 0.8661081800129524
tau_perturbed_e_t[i]: 


prob.status: optimal

welfare_at_e_t: 8196.83644870212
obj_E_at_e_t: 465.7084332004423
obj_R_at_e_t: 635.088839362033
obj_I_at_e_t: 8366.216854863711

welfare_perturbed_at_e_t: 8196.827673679387
obj_E_perturbed_at_e_t: 465.70957391350396
obj_R_perturbed_at_e_t: 664.1686534188407
obj_I_perturbed_at_e_t: 8395.286753184724

tau_e_t[:, :, i+1], before projection:
 0.5281607942222555
alpha_e_t[:, :, i+1], before projection:
 0.8479060901175173

tau_feas.value: [0.528]

alpha_feas: 0.8479060901175173

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.528]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.848
tau_e_t[:, :, i+1], after projection:
 0.528
alpha_e_t[:, :, i+1], after projection:
 0.848
tau_diffs:
 0.006000000000000005
alpha_diffs:
 0.006000000000000005

Within error bound.
e: 0
t: 1

Iter: 0
i:
 0
tau_e_t[i], before projection:
 0.28
alpha_e_t[i], before projection:
 0.876894067600577
tau_perturbed_e_t[i], before projection:
 0.3141111232627987
alpha_perturbed_e_t[i]


prob.status: optimal

welfare_at_e_t: 7938.871341569068
obj_E_at_e_t: 457.92550745985574
obj_R_at_e_t: 336.9408884343098
obj_I_at_e_t: 7817.886722543522

welfare_perturbed_at_e_t: 7938.872642424023
obj_E_perturbed_at_e_t: 457.92516958121763
obj_R_perturbed_at_e_t: 301.7935313821289
obj_I_perturbed_at_e_t: 7782.741004224934

tau_e_t[:, :, i+1], before projection:
 0.27876699899573465
alpha_e_t[:, :, i+1], before projection:
 0.8810506836604414

tau_feas.value: [0.279]

alpha_feas: 0.8810506836604414

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.279]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.881
tau_e_t[:, :, i+1], after projection:
 0.279
alpha_e_t[:, :, i+1], after projection:
 0.881

Iteration count: 3
Time: 0.31037020683288574


Iter: 4
i:
 4
tau_e_t[i], before projection:
 0.279
alpha_e_t[i], before projection:
 0.881
tau_perturbed_e_t[i], before projection:
 0.26019079129664047
alpha_perturbed_e_t[i], before projection:
 0.8376147300866613
tau_perturbed_e_t


prob.status: optimal

welfare_at_e_t: 7938.871674966178
obj_E_at_e_t: 457.92541945751395
obj_R_at_e_t: 329.66910091468395
obj_I_at_e_t: 7810.615356423348

welfare_perturbed_at_e_t: 7938.872724902942
obj_E_perturbed_at_e_t: 457.9251480285456
obj_R_perturbed_at_e_t: 376.9389966499586
obj_I_perturbed_at_e_t: 7857.886573524354

tau_e_t[:, :, i+1], before projection:
 0.2711733814340934
alpha_e_t[:, :, i+1], before projection:
 0.8853101500835251

tau_feas.value: [0.271]

alpha_feas: 0.8853101500835251

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.271]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.885
tau_e_t[:, :, i+1], after projection:
 0.271
alpha_e_t[:, :, i+1], after projection:
 0.885
tau_diffs:
 0.006000000000000005
alpha_diffs:
 0.0030000000000000027

Within error bound.
e: 0
t: 2

Iter: 0
i:
 0
tau_e_t[i], before projection:
 0.46
alpha_e_t[i], before projection:
 0.8685475232899137
tau_perturbed_e_t[i], before projection:
 0.454660790052545
alpha_perturbed_e_

...


prob.status: optimal
coeff:
 [[1.32644825e+00 2.21312655e+00 4.89219237e+00 1.19991118e+00
  5.54100628e+00 1.50311134e+00 2.38432845e+00]
 [7.82666000e-04 5.84484000e-04 1.56372400e-03 1.99413800e-03
  2.14726200e-03 3.06601000e-04 3.21856000e-04]
 [8.61988500e+02 1.00151786e+03 8.81184667e+02 1.27894812e+03
  1.03409283e+03 8.45150000e+02 8.53181818e+02]]


prob.status: optimal

welfare_at_e_t: 16885.970609264576
obj_E_at_e_t: 1282.431752692174
obj_R_at_e_t: 844.6995685541827
obj_I_at_e_t: 16448.238425126583

welfare_perturbed_at_e_t: 16885.971973550557
obj_E_perturbed_at_e_t: 1282.4311886910757
obj_R_perturbed_at_e_t: 819.808286632171
obj_I_perturbed_at_e_t: 16423.34907149165

tau_e_t[:, :, i+1], before projection:
 0.47539334291852064
alpha_e_t[:, :, i+1], before projection:
 0.8849837037923778

tau_feas.value: [0.475]

alpha_feas: 0.8849837037923778

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.475]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.885
tau_e_t[:, 


prob.status: optimal
coeff:
 [[1.32644825e+00 2.21312655e+00 4.89219237e+00 1.19991118e+00
  5.54100628e+00 1.50311134e+00 2.38432845e+00]
 [7.82666000e-04 5.84484000e-04 1.56372400e-03 1.99413800e-03
  2.14726200e-03 3.06601000e-04 3.21856000e-04]
 [8.61988500e+02 1.00151786e+03 8.81184667e+02 1.27894812e+03
  1.03409283e+03 8.45150000e+02 8.53181818e+02]]


prob.status: optimal

welfare_at_e_t: 17810.67226307561
obj_E_at_e_t: 1092.5063721407514
obj_R_at_e_t: 705.9371934428044
obj_I_at_e_t: 17424.10308437766

welfare_perturbed_at_e_t: 17810.69217358192
obj_E_perturbed_at_e_t: 1092.501073301459
obj_R_perturbed_at_e_t: 787.7869193642357
obj_I_perturbed_at_e_t: 17505.978019644695

tau_e_t[:, :, i+1], before projection:
 0.3786698957572268
alpha_e_t[:, :, i+1], before projection:
 0.9039653633721917

tau_feas.value: [0.379]

alpha_feas: 0.9039653633721917

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.379]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.904
tau_e_t[:, :,


prob.status: optimal
coeff:
 [[1.32644825e+00 2.21312655e+00 4.89219237e+00 1.19991118e+00
  5.54100628e+00 1.50311134e+00 2.38432845e+00]
 [7.82666000e-04 5.84484000e-04 1.56372400e-03 1.99413800e-03
  2.14726200e-03 3.06601000e-04 3.21856000e-04]
 [8.61988500e+02 1.00151786e+03 8.81184667e+02 1.27894812e+03
  1.03409283e+03 8.45150000e+02 8.53181818e+02]]


prob.status: optimal

welfare_at_e_t: 17810.695409040323
obj_E_at_e_t: 1092.5001627619715
obj_R_at_e_t: 663.3140157915192
obj_I_at_e_t: 17381.50926206987

welfare_perturbed_at_e_t: 17810.691972175016
obj_E_perturbed_at_e_t: 1092.5011305037026
obj_R_perturbed_at_e_t: 615.2912312131007
obj_I_perturbed_at_e_t: 17333.482072884413

tau_e_t[:, :, i+1], before projection:
 0.37112066914524283
alpha_e_t[:, :, i+1], before projection:
 0.9043231833456804

tau_feas.value: [0.371]

alpha_feas: 0.9043231833456804

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.371]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.904
tau_e_t[:


prob.status: optimal
coeff:
 [[1.32644825e+00 2.21312655e+00 4.89219237e+00 1.19991118e+00
  5.54100628e+00 1.50311134e+00 2.38432845e+00]
 [7.82666000e-04 5.84484000e-04 1.56372400e-03 1.99413800e-03
  2.14726200e-03 3.06601000e-04 3.21856000e-04]
 [8.61988500e+02 1.00151786e+03 8.81184667e+02 1.27894812e+03
  1.03409283e+03 8.45150000e+02 8.53181818e+02]]


prob.status: optimal

welfare_at_e_t: 18139.963508196666
obj_E_at_e_t: 1219.5423205967643
obj_R_at_e_t: 947.8469140568565
obj_I_at_e_t: 17868.26810165676

welfare_perturbed_at_e_t: 18139.959939947385
obj_E_perturbed_at_e_t: 1219.5432542447977
obj_R_perturbed_at_e_t: 896.2659892307835
obj_I_perturbed_at_e_t: 17816.68267493337

tau_e_t[:, :, i+1], before projection:
 0.5309248256225746
alpha_e_t[:, :, i+1], before projection:
 0.8662986504870295

tau_feas.value: [0.531]

alpha_feas: 0.8662986504870295

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.531]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.866
tau_e_t[:, 


prob.status: optimal
coeff:
 [[1.32644825e+00 2.21312655e+00 4.89219237e+00 1.19991118e+00
  5.54100628e+00 1.50311134e+00 2.38432845e+00]
 [7.82666000e-04 5.84484000e-04 1.56372400e-03 1.99413800e-03
  2.14726200e-03 3.06601000e-04 3.21856000e-04]
 [8.61988500e+02 1.00151786e+03 8.81184667e+02 1.27894812e+03
  1.03409283e+03 8.45150000e+02 8.53181818e+02]]


prob.status: optimal

welfare_at_e_t: 18139.963063233718
obj_E_at_e_t: 1219.542436810116
obj_R_at_e_t: 947.8456571128432
obj_I_at_e_t: 17868.266283536446

welfare_perturbed_at_e_t: 18139.959218047312
obj_E_perturbed_at_e_t: 1219.543443099941
obj_R_perturbed_at_e_t: 997.627254652069
obj_I_perturbed_at_e_t: 17918.04302959944

tau_e_t[:, :, i+1], before projection:
 0.5351558722916562
alpha_e_t[:, :, i+1], before projection:
 0.8632752080868161

tau_feas.value: [0.533]

alpha_feas: 0.8632752080868161

np.round(np.maximum(tau_feas.value, 0.0), decimals=3): [0.533]
np.round(np.maximum(alpha_feas, 0.0), decimals=3): 0.863
tau_e_t[:, :,

In [78]:
obj_DBCP_init_min_as_array = np.zeros((num_edges, T))

for e in range(num_edges):
    for t in range(T):
        obj_DBCP_init_min_as_array[e, t] = obj_DBCP_init_min[(e, t)]
        
obj_DBCP_init_min_as_array

array([[ 8598.71631722,  7931.54335948,  7933.62871918,  8037.24559109,
         9041.01516518],
       [13230.70178873, 11596.28385888, 12527.4540351 , 12152.81075029,
        13038.54872247],
       [40750.97807486, 39729.82914462, 39279.15717313, 40038.44909205,
        42234.60463397],
       [ 9651.00783807,  9640.31314478, 10905.83238819,  9706.68495041,
        13488.89376942],
       [57888.21879072, 57186.23716838, 55408.14606982, 56698.33142517,
        58406.95943063],
       [11533.05817775, 11248.59945858, 10800.48386858, 11013.63113071,
        11242.33132325],
       [19115.02904229, 18018.29968078, 17166.2478501 , 17812.45515211,
        18511.33620611]])

In [79]:
np.all(welfare_min_array <= obj_DBCP_init_min_as_array)

True

In [80]:
print("Time:", time_2 - time_1)

Time: 188.76343822479248


In [81]:
argmin_tau

array([[0.92, 0.28, 0.46, 0.48, 0.94],
       [0.95, 0.32, 0.8 , 0.58, 0.7 ],
       [0.51, 2.23, 2.08, 1.35, 2.19],
       [1.27, 0.86, 1.72, 1.38, 2.47],
       [2.13, 2.29, 2.25, 0.58, 1.31],
       [0.53, 0.44, 0.69, 0.5 , 0.38],
       [0.79, 0.26, 0.47, 0.36, 0.75]])

In [82]:
# argmin_tau_new 
argmin_tau_new - argmin_tau

array([[-3.870e-01,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00],
       [-4.410e-01,  0.000e+00, -3.280e-01, -7.100e-02, -1.670e-01],
       [ 0.000e+00, -1.740e+00, -1.552e+00,  0.000e+00, -1.657e+00],
       [ 0.000e+00, -3.270e-01, -1.187e+00,  0.000e+00, -2.013e+00],
       [-1.597e+00, -1.938e+00, -1.939e+00,  0.000e+00, -9.060e-01],
       [ 2.000e-03, -9.000e-03, -1.590e-01,  0.000e+00, -4.000e-03],
       [-2.570e-01, -2.000e-03,  0.000e+00,  1.200e-02,  0.000e+00]])

In [83]:
init_alpha

array([[0.85596331, 0.87689407, 0.86854752, 0.85819808, 0.85440341],
       [0.8454951 , 0.87720412, 0.85076804, 0.85581165, 0.8516927 ],
       [0.86840947, 0.87852273, 0.86854752, 0.88331776, 0.8627252 ],
       [0.85557171, 0.9449055 , 0.88818808, 0.88331776, 0.85050779],
       [0.87957513, 0.88477124, 0.88944531, 0.88519769, 0.94608763],
       [0.85596331, 0.88477124, 0.85351709, 0.88543049, 0.8807204 ],
       [0.85596331, 0.88702193, 0.86830451, 0.88841123, 0.87089765]])

In [84]:
argmin_alpha_new 
# argmin_alpha_new - init_alpha

array([[0.853     , 0.87689407, 0.86854752, 0.85819808, 0.85440341],
       [0.923     , 0.87720412, 0.901     , 0.904     , 0.894     ],
       [0.86840947, 0.77      , 0.889     , 0.88331776, 0.885     ],
       [0.85557171, 0.856     , 0.882     , 0.88331776, 0.785     ],
       [0.891     , 0.959     , 0.951     , 0.88519769, 0.967     ],
       [0.862     , 0.893     , 0.851     , 0.88543049, 0.95      ],
       [0.855     , 0.884     , 0.886     , 0.883     , 0.87089765]])

In [85]:
welfare_min_array

array([[ 8196.83521466,  7931.54335948,  7933.62871918,  8037.24559109,
         8453.06967428],
       [12017.30558871, 11596.28385888, 11638.09374581, 12021.59291534,
        12459.05556206],
       [40750.97807486, 39416.16291929, 39218.60717458, 40038.44909205,
        42025.47055491],
       [ 9651.00783807,  9242.22650099,  9212.59674373,  9706.68495041,
         9930.77311143],
       [57772.70137808, 55708.37598408, 54172.43863169, 56698.33142517,
        57568.47298668],
       [11531.58340174, 10993.87103885, 10474.9584379 , 11013.63113071,
        11027.75370216],
       [18852.46268291, 17896.1158882 , 16885.96992795, 17810.66257585,
        18139.95925852]])

In [86]:
# np.all(welfare_min_array <= obj_DBCP_init_min_as_array)

welfare_min_array - obj_DBCP_init_min_as_array

array([[-4.01881103e+02,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00, -5.87945491e+02],
       [-1.21339620e+03,  0.00000000e+00, -8.89360289e+02,
        -1.31217835e+02, -5.79493160e+02],
       [ 0.00000000e+00, -3.13666225e+02, -6.05499985e+01,
         0.00000000e+00, -2.09134079e+02],
       [ 0.00000000e+00, -3.98086644e+02, -1.69323564e+03,
         0.00000000e+00, -3.55812066e+03],
       [-1.15517413e+02, -1.47786118e+03, -1.23570744e+03,
         0.00000000e+00, -8.38486444e+02],
       [-1.47477602e+00, -2.54728420e+02, -3.25525431e+02,
         0.00000000e+00, -2.14577621e+02],
       [-2.62566359e+02, -1.22183793e+02, -2.80277922e+02,
        -1.79257626e+00, -3.71376948e+02]])

In [87]:
y_opt_DBCP_new

{(3, 3, 0, 0, 0): -0.0,
 (3, 3, 0, 1, 0): 56.29281030872774,
 (1, 3, 0, 0, 0): -0.0,
 (1, 3, 0, 1, 0): 21.567682369940975,
 (4, 3, 0, 0, 0): -0.0,
 (4, 3, 0, 1, 0): 213.4139705212124,
 (5, 3, 0, 0, 0): -0.0,
 (5, 3, 0, 1, 0): 53.353241347104095,
 (2, 3, 0, 0, 0): 250.36177510425878,
 (2, 3, 0, 1, 0): -0.0,
 (0, 3, 0, 0, 0): 134.75871572598552,
 (0, 3, 0, 1, 0): 2.842170943040401e-14,
 (6, 3, 0, 0, 0): 108.8087666776787,
 (6, 3, 0, 1, 0): 228.02583709324324,
 (4, 4, 0, 0, 0): -0.0,
 (4, 4, 0, 1, 0): 454.9597826111301,
 (5, 4, 0, 0, 0): -0.0,
 (5, 4, 0, 1, 0): 113.7394099626901,
 (0, 4, 0, 0, 0): -0.0,
 (0, 4, 0, 1, 0): 287.28108034312373,
 (2, 4, 0, 0, 0): -0.0,
 (2, 4, 0, 1, 0): 533.7257841995335,
 (3, 4, 0, 0, 0): -0.0,
 (3, 4, 0, 1, 0): 120.00603652178778,
 (1, 4, 0, 0, 0): -0.0,
 (1, 4, 0, 1, 0): 45.978377415919624,
 (6, 4, 0, 0, 0): 718.0701325843745,
 (6, 4, 0, 1, 0): -0.0,
 (0, 0, 0, 0, 0): -0.0,
 (0, 0, 0, 1, 0): 47.77809012103125,
 (6, 0, 0, 0, 0): -0.0,
 (6, 0, 0, 1, 0): 119.4

In [88]:
# obj_E_min_at_e_t

In [89]:


# # print("argmin_B:", argmin_B)
# # print("B[argmin_welfare_list]:", B[argmin_welfare_list])
# # print("min_welfare:", min_welfare)
# # print()
# # print(welfare_list)

# # B
# print()
# print(min_welfare_at_e_t)
# print()
# print(argmin_welfare_list_at_e_t)
# print()
# print("len(welfare_list_at_e_t):\n", len(welfare_list_at_e_t))
# print()
# print("argmin_tau_at_e_t:\n", argmin_tau_at_e_t)
# print()
# print("argmin_alpha_at_e_t:\n", argmin_alpha_at_e_t)

# # welfare_list[argmin_welfare_list]
# # B[argmin_welfare_list]

## (August 13) Storing data

In [115]:
# Change argmin_y format from dict to array:

# y_opt_DBCP_new


y_opt_DBCP_new_array = np.zeros((len(list(y_opt_DBCP_new.keys())) , 6))

for row_index, key in enumerate(list(y_opt_DBCP_new.keys())):
#     print(key)
    
    y_opt_DBCP_new_array[row_index, :5] = np.array(key)
    
    if y_opt_DBCP_new[key] <= 0.0 and y_opt_DBCP_new_array[row_index, -1] >= -1E-3:
        y_opt_DBCP_new_array[row_index, -1] = 0.0
    else:
        y_opt_DBCP_new_array[row_index, -1] = y_opt_DBCP_new[key]
#     print("row_index, key", (row_index, key))



In [116]:
directory_path = "../data/opt_values___3_el_groups/"

if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
    str_int_lambda_E = str(int(lambda_E))
else:
    str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
    str_int_lambda_R = str(int(lambda_R))
else:
    str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
    str_int_lambda_I = str(int(lambda_I))
else:
    str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I

filename = filename_segment + "___y_DBCP.csv"


column_names = ["od", "g", "e", "k", "t", "flows (y)"]

df_y_opt_DBCP_new = pd.DataFrame(y_opt_DBCP_new_array, columns = column_names)
df_y_opt_DBCP_new["od"] = df_y_opt_DBCP_new["od"].astype(int)
df_y_opt_DBCP_new["g"] = df_y_opt_DBCP_new["g"].astype(int)
df_y_opt_DBCP_new["e"] = df_y_opt_DBCP_new["e"].astype(int)
df_y_opt_DBCP_new["k"] = df_y_opt_DBCP_new["k"].astype(int)
df_y_opt_DBCP_new["t"] = df_y_opt_DBCP_new["t"].astype(int)

df_y_opt_DBCP_new.to_csv(directory_path + filename, index=False)

df_y_opt_DBCP_new

# y_opt_DBCP_new


,od,g,e,k,t,flows (y)
0,3,3,0,0,0,0.000000
1,3,3,0,1,0,56.292810
2,1,3,0,0,0,0.000000
3,1,3,0,1,0,21.567682
4,4,3,0,0,0,0.000000
...,...,...,...,...,...,...
3995,23,2,6,1,4,263.686411
3996,11,2,6,0,4,0.000046
3997,11,2,6,1,4,115.319702
3998,6,2,6,0,4,0.000051


In [96]:
# y_agg_DBCP_new
# y_opt_DBCP_new

In [97]:
y_agg_DBCP_new, x_DBCP_new = \
    compute_y_agg_x(y = y_opt_DBCP_new, \
                    edge_to_od_dict = edge_to_od_dict, \
                    num_edges = num_edges, \
                    T = T)

travel_times_DBCP_new, avg_travel_times_DBCP_new = \
    compute_travel_times(y_agg = y_agg_DBCP_new, \
                         x = x_DBCP_new, \
                         edge_to_od_dict = edge_to_od_dict, \
                         coeff_input = coeff_input, \
                         num_gp_lanes = num_gp_lanes, \
                         num_edges = num_edges, \
                         T = T)

percent_on_express_DBCP_new = \
    compute_percent_on_express(y_agg = y_agg_DBCP_new, \
                               x = x_DBCP_new, \
                               num_edges = num_edges, \
                               T = T)
    
    
    

In [104]:
avg_travel_times_DBCP_new

array([1306.08270559, 1147.57209242, 1282.43203685, 1092.50904017,
       1219.54343522])

In [109]:
opt_data_array = np.zeros((num_edges, 20))

# argmin_tau
opt_data_array[:, 0:5] = argmin_tau_new

# argmin_tau_avg
opt_data_array[:, 5] = np.mean(argmin_tau_new, axis=1)

# argmin_alpha
opt_data_array[:, 6:11] = argmin_alpha_new

# percent_on_express (overall)
# percent_on_express (eligible)
# percent_on_express (ineligible)
opt_data_array[:, 11] = np.array([percent_on_express_DBCP_new[e, 'all'] for e in range(num_edges)]) * 100
opt_data_array[:, 12] = np.array([percent_on_express_DBCP_new[e, 'el'] for e in range(num_edges)]) * 100
opt_data_array[:, 13] = np.array([percent_on_express_DBCP_new[e, 'in'] for e in range(num_edges)]) * 100

# avg_travel_time (express lane)
# avg_travel_time (general purpose lane)
opt_data_array[:, 14] = np.array([avg_travel_times_DBCP_new[e, 'ex'] for e in range(num_edges)])
opt_data_array[:, 15] = np.array([avg_travel_times_DBCP_new[e, 'gp'] for e in range(num_edges)])

# obj_E = {}
# obj_I = {}
# obj_R = {}
# obj
opt_data_array[:, 16] = np.array([np.sum(obj_E_min_at_e_t[e, :]) for e in range(num_edges)])
opt_data_array[:, 17] = np.array([np.sum(obj_I_min_at_e_t[e, :]) for e in range(num_edges)])
opt_data_array[:, 18] = np.array([np.sum(obj_R_min_at_e_t[e, :]) for e in range(num_edges)])
opt_data_array[:, 19] = np.array([np.sum(welfare_min_array[e, :]) for e in range(num_edges)])

opt_data_array = np.round(opt_data_array, decimals=2)

In [110]:
column_names = []
column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
column_names += ["tau (time-averaged)"]
column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]
# column_names += ["alpha (time-averaged)"]
column_names += ["% overall users using express lanes", \
                 "% eligible users using express lanes", \
                 "% ineligible users using express lanes", \
                 "Average travel time (express lanes)", \
                 "Average travel time (general purpose lanes)", \
                 "Total travel cost (eligible users)", \
                 "Total travel cost (ineligible users)", \
                 "Total toll revenue", \
                 "Total societal cost"]

row_names = ["e=" + str(k+1) for k in range(num_edges) ]

df_opt_DBCP_save = pd.DataFrame(opt_data_array, index=row_names, columns=column_names)

df_opt_DBCP_save

,tau (t=1),tau (t=2),tau (t=3),tau (t=4),tau (t=5),tau (time-averaged),alpha (t=1),alpha (t=2),alpha (t=3),alpha (t=4),alpha (t=5),% overall users using express lanes,% eligible users using express lanes,% ineligible users using express lanes,Average travel time (express lanes),Average travel time (general purpose lanes),Total travel cost (eligible users),Total travel cost (ineligible users),Total toll revenue,Total societal cost
e=1,0.53,0.28,0.46,0.48,0.94,0.54,0.85,0.88,0.87,0.86,0.85,22.64,14.11,26.49,1.51,1.63,2308.64,40920.42,2676.74,40552.32
e=2,0.51,0.32,0.47,0.51,0.53,0.47,0.92,0.88,0.90,0.90,0.89,24.23,6.34,33.50,2.36,2.39,4022.71,58519.27,2809.65,59732.33
e=3,0.51,0.49,0.53,1.35,0.53,0.68,0.87,0.77,0.89,0.88,0.88,23.72,11.19,30.12,6.03,6.22,13540.92,192237.21,4328.46,201449.67
e=4,1.27,0.53,0.53,1.38,0.46,0.83,0.86,0.86,0.88,0.88,0.78,23.18,8.01,30.62,1.48,1.74,3222.11,49182.84,4661.66,47743.29
e=5,0.53,0.35,0.31,0.58,0.40,0.44,0.89,0.96,0.95,0.89,0.97,24.75,8.44,32.85,7.54,7.60,18868.02,266754.11,3701.80,281920.32
e=6,0.53,0.43,0.53,0.50,0.38,0.47,0.86,0.89,0.85,0.89,0.95,23.21,3.19,33.04,1.72,1.77,3686.47,54840.82,3485.49,55041.80
e=7,0.53,0.26,0.47,0.37,0.75,0.48,0.86,0.88,0.89,0.88,0.87,25.00,0.00,37.26,2.68,2.68,6048.14,87774.66,4237.63,89585.17


In [111]:
directory_to_save = "../data/opt_values___3_el_groups/"
# filename = "opt_CBCP_params___" + random_string + '.csv'

if lambda_E >= 1.0 - 1E-3 or lambda_E <= 1E-3:
    str_int_lambda_E = str(int(lambda_E))
else:
    str_int_lambda_E = 'point_' + str(int(lambda_E * 100))

if lambda_R >= 1.0 - 1E-3 or lambda_R <= 1E-3:
    str_int_lambda_R = str(int(lambda_R))
else:
    str_int_lambda_R = 'point_' + str(int(lambda_R * 100))
    
if lambda_I >= 1.0 - 1E-3 or lambda_I <= 1E-3:
    str_int_lambda_I = str(int(lambda_I))
else:
    str_int_lambda_I = 'point_' + str(int(lambda_I * 100))
    
filename_segment = str_int_lambda_E + '_' + str_int_lambda_R + '_' + str_int_lambda_I

filename = filename_segment + '___tau_B_stats_DBCP.csv'

df_opt_DBCP_save.to_csv(directory_to_save + filename)

In [ ]:
argmin_tau

In [ ]:
init_alpha

In [ ]:
# # y_agg_el_at_e_t
# print("y_agg_el_ex:", y_agg_el_ex)

# e_temp = 1
# sum([demand_array[od, g] for od in edge_to_od_dict[e_temp] for g in el_indices])

In [ ]:
# y_DBCP_init_at_e_t

# assert y_DBCP_init_at_e_t == extract_y_at_e_t(y_DBCP_init, 6, 4)

In [ ]:
# for e in range(num_edges):
#     print("Edge:", e)
    
#     assert equals(obj_DBCP_init_together[(e, 'E')], obj_DBCP_init[(e, 'E')])
#     assert equals(obj_DBCP_init_together[(e, 'R')], obj_DBCP_init[(e, 'R')])
#     assert equals(obj_DBCP_init_together[(e, 'I')], obj_DBCP_init[(e, 'I')])
    

# Old Code

In [ ]:
# list_1 = [1, 2, 3]
# list_2 = [2, 3, 4]

# [x for x in list_1 if x not in list_2]


In [ ]:
edge_to_od_dict

In [ ]:
# init_DBCP_key_list_comp = [(od, g, e, k, t) for e in range(num_edges) for t in range(T) \
#                               for k in [0, 1] for g in (el_indices + in_indices) for od in edge_to_od_dict[e]]

# list(init_DBCP_flows.keys()) == init_DBCP_key_list_comp

# keys_missing_init_DBCP_flows = [key for key in list(init_DBCP_flows.keys()) \
#                                 if key not in init_DBCP_key_list_comp]
# keys_extra_init_DBCP_flows = [key for key in init_DBCP_key_list_comp \
#                                 if key not in list(init_DBCP_flows.keys())]

# print("keys_missing_init_DBCP_flows:", keys_missing_init_DBCP_flows)
# print("\n")
# print("keys_extra_init_DBCP_flows:", keys_extra_init_DBCP_flows)

In [ ]:
y_in_el_total_init_DBCP = {}
x_init_DBCP = {}

for e in range(num_edges):
    for t in range(T):
        for k in [0, 1]:
            y_in_el_total_init_DBCP[(e, k, t, "in")] \
                = sum(init_DBCP_flows[(od, g, e, k, t)] for od in edge_to_od_dict[e] for g in in_indices)
            y_in_el_total_init_DBCP[(e, k, t, "el")] \
                = sum(init_DBCP_flows[(od, g, e, k, t)] for od in edge_to_od_dict[e] for g in el_indices)
            
            x_init_DBCP[(e, k, t)] = y_in_el_total_init_DBCP[(e, k, t, "in")] \
                                        + y_in_el_total_init_DBCP[(e, k, t, "el")]


In [ ]:
y_init_DBCP

In [ ]:
y_in_el_total_init_DBCP

In [ ]:
x_init_DBCP

In [ ]:
# lambda_R = 1

travel_times_DBCP = {}

## coeff_input: const, slope, x-coordinate of transition point
# coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))

for e in range(num_edges):
    for t in range(T):
        travel_times_DBCP[(e, 0, t)] \
            = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 0, t)] - coeff_input[2, e], 0.0)
        travel_times_DBCP[(e, 1, t)] \
            = coeff_input[0, e] + coeff_input[1, e] * max(argmin_x[(e, 1, t)]/num_gp_lanes - coeff_input[2, e], 0.0)

# INCOMPLETE:
        
edge_demand_DBCP = {}
avg_travel_time_DBCP = {}
percent_on_express_DBCP = {}
obj_E_DBCP = np.zeros(num_edges)
obj_I_DBCP = np.zeros(num_edges)
obj_R_DBCP = np.zeros(num_edges)
obj_DBCP = np.zeros(num_edges)

for e in range(num_edges):
    avg_travel_time_DBCP[e, 'el'] = 0.0
    avg_travel_time_DBCP[e, 'in'] = 0.0
    avg_travel_time_DBCP[e, 'ex'] = 0.0
    avg_travel_time_DBCP[e, 'gp'] = 0.0
    
    percent_on_express_DBCP[e, 'el'] = 0.0
    percent_on_express_DBCP[e, 'in'] = 0.0
    percent_on_express_DBCP[e, 'all'] = 0.0
    
    obj_E_DBCP[e] = 0.0
    obj_I_DBCP[e] = 0.0
    obj_R_DBCP[e] = 0.0
    obj_DBCP[e] = 0.0
    
    for t in range(T):
        edge_demand_DBCP[e, t, 'el'] = sum(argmin_y_in_el_total[(e, k, t, 'el')] for k in range(2))
        edge_demand_DBCP[e, t, 'in'] = sum(argmin_y_in_el_total[(e, k, t, 'in')] for k in range(2))
        
    percent_on_express_DBCP[e, 'el'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] for t in range(T))
    percent_on_express_DBCP[e, 'in'] += sum(argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'in'] for t in range(T))
    percent_on_express_DBCP[e, 'all'] += sum(argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] + edge_demand_DBCP[e, t, 'in'] for t in range(T))
    
    avg_travel_time_DBCP[e, 'el'] += sum(argmin_y_in_el_total[(e, k, t, 'el')] * travel_times_DBCP[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'el'] for t in range(T))    
    avg_travel_time_DBCP[e, 'in'] += sum(argmin_y_in_el_total[(e, k, t, 'in')] * travel_times_DBCP[(e, k, t)] for k in range(2) for t in range(T)) \
                                    / sum(edge_demand_DBCP[e, t, 'in'] for t in range(T))    
#     avg_travel_time_DBCP[e, 'ex'] += sum( (argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')]) * travel_times_DBCP[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 0, t, 'el')] + argmin_y_in_el_total[(e, 0, t, 'in')] for t in range(T))
#     avg_travel_time_DBCP[e, 'gp'] += sum( (argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')]) * travel_times_DBCP[(e, 0, t)] for t in range(T)) \
#                                     / sum( argmin_y_in_el_total[(e, 1, t, 'el')] + argmin_y_in_el_total[(e, 1, t, 'in')] for t in range(T))
    avg_travel_time_DBCP[e, 'ex'] += sum(travel_times_DBCP[(e, 0, t)] for t in range(T)) / T 
    avg_travel_time_DBCP[e, 'gp'] += sum(travel_times_DBCP[(e, 1, t)] for t in range(T)) / T 
    
    obj_E_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) * (1 - init_alpha[e, t]) \
                    + sum( y_init_DBCP[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 1, t] \
                          for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) 
    
#                     + sum( y_init_DBCP[(od, g, e, 1, t)] * (VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] + argmin_tau[e, t]) \
#                         for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) \

    obj_I_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * (VoT_array[od, g, t] * travel_times_DBCP[e, 0, t] + argmin_tau[e, t]) \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 1, t)] * VoT_array[od, g, t] * travel_times_DBCP[e, 1, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) )

    obj_R_DBCP[e] = sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in in_indices for t in range(T) ) \
                    + sum( y_init_DBCP[(od, g, e, 0, t)] * argmin_tau[e, t] \
                        for od in edge_to_od_dict[e] for g in el_indices for t in range(T) ) * (1 - init_alpha[e, t])

    obj_DBCP[e] = lambda_E * obj_E_DBCP[e] - lambda_R * obj_R_DBCP[e] + lambda_I * obj_I_DBCP[e]


# welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, argmin_tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input)

# avg_travel_time_DBCP
# percent_on_express_DBCP
# obj_R_DBCP

sum(obj_DBCP)

In [ ]:
# column_names = []
# column_names += ["tau (t=" + str(t+1) + ")" for t in range(T) ]
# column_names += ["alpha (t=" + str(t+1) + ")" for t in range(T) ]

# row_names = ["e=" + str(k+1) for k in range(num_edges) ]

# df_inits_save = pd.DataFrame(init_tau_alpha_array, index=row_names, columns=column_names)

# # df_inits_save

# # random_string = ""
# # for idx_rand_str in range(10):
# #     random_string += str(np.random.randint(1, 9))

# directory_to_save = "../data/old___opt_tolls_subsidies_metrics/"
# random_filename = "inits___" + random_string + '.csv'

# df_inits_save.to_csv(directory_to_save + random_filename)

In [ ]:
print("first(welfares):", welfares[0])
print("min(welfares):", min(welfares))
print("max(welfares):", max(welfares))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
